In [ ]:
from IPython.display import HTML, display

HTML('''<script>
code_show=true;
function code_toggle() {
  if (code_show){
  $('div.input').hide();
  } else {
  $('div.input').show();
  }
  code_show = !code_show
}
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()">
<input type="submit" value="Reveal/Hide Code"></form>''')

In [ ]:
# # Environment creation commands:
# conda create -y -n py3 python=3.7
# conda activate py3
# conda install -y nb_conda
# pip install altair vega_datasets altair_saver selenium
# pip install matplotlib seaborn datetime
# pip install python-pptx
# pip install pycountry_convert requests
# # Check version of ChromeDriver according to version of Chrome
# wget https://chromedriver.storage.googleapis.com/87.0.4280.20/chromedriver_mac64.zip
# unzip chromedriver_mac64.zip
# rm chromedriver_mac64.zip
# mv chromedriver /usr/local/bin/

# jupyter notebook covid.ipynb

# # Environment removal after the test
# conda deactivate py3
# conda env remove -y -n py3

# # Would like to be able to use texture (patterns) to help interpretation or add further information
# # can draw texture using svg, but not yet managed to save to image file, so not yet working with automatic slide generation flow
# https://github.com/altair-viz/altair/issues/2103

In [ ]:
# Package imports
import os
import re

import altair as alt
from altair_saver import save
from vega_datasets import data

import json
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

import datetime

import pycountry_convert
from pycountry_convert import country_alpha2_to_continent_code

import requests
from urllib.request import urlopen
from http import HTTPStatus

# Package configurations
alt.data_transformers.disable_max_rows()

# Configuration for notebook
aggregate_states = True # option to roll-up multiple states to a top level per country view
show_charts_for_all_countries = False # option to show full long list of charts for all countries
num_cases_threshold = 1500 # optional number of cases threshold to filter out what would otherwise be 'noisy' statistics; value is None, or integer threashold
countries_to_aggregate = ['Australia', 'Canada', 'China']

run_date_str = "{:%d-%b-%Y}".format(datetime.datetime.now())

# Slide export configuration
export_slides = True
slides_file_name = 'COVID_Summary.pptx'
template_pptx = 'widescreen.pptx' # input slide template, to help set aspect ratio
saved_image_name = 'image.png'

In [ ]:
def display_h1(myString):
    """display as heading 1"""

    display(HTML('<h1>%s</h1>' % myString))

def display_h2(myString):
    """display as heading 2"""

    display(HTML('<h2>%s</h2>' % myString))

def display_table(myDF):
    """display as formatted table"""

    total_id = 'totalID'
    header_id = 'headerID'
    data_in_html = myDF.to_html(index=False)
    tr_style = '<style>.dataframe td { text-align: left; }</style>'
    style_in_html = """<style>
        table#{total_table} {{color='black';font-size:13px; text-align:center; border:0.2px solid black;
                             border-collapse:collapse; max-width: 900px; word-wrap:break-word; white-space:pre-wrap; height='250'; }}
        thead#{header_table} {{background-color: #4D4D4D; color:#ffffff}}
        </style>""".format(total_table=total_id, header_table=header_id)
    data_in_html = re.sub(r'<table',
                          r'<table id=%s ' % total_id, data_in_html)
    data_in_html = re.sub(r'<thead', r'<thead id=%s ' % header_id, data_in_html)

    display(HTML(style_in_html + tr_style + data_in_html))

def display_set_full_width():
    """use the full screen width"""

    display(HTML("""
    <style>
        div#notebook-container    { width: 100%; }
        div#menubar-container     { width: 65%; }
        div#maintoolbar-container { width: 99%; }
    </style>
    """))
    
display_set_full_width()

In [ ]:
def df_remove_trailing_zeros(df, metric):
    df_last_index_list = df[df[metric] != 0]
    if len(df_last_index_list):
        df_last_index = df_last_index_list.index[-1]
        df = df[:df_last_index]

    return df

def df_delta_from_cumulative(df, metric, category, new_metric):
    if category:
        df = df.sort_values(by=['date', category], ascending=True)
        last_date = df.groupby([category]).apply(lambda x: x.iloc[-1]).iloc[0]['date']
        penultimate_date = df.groupby([category]).apply(lambda x: x.iloc[-2]).iloc[0]['date']
        if last_date > penultimate_date:
            step = +1
        else:
            step = -1

        df[new_metric] = df.groupby([category])[metric].transform(lambda x: x.diff(periods=step)).clip(lower=0).fillna(0)
    else:
        df = df.sort_values(by=['date'], ascending=True)

        last_date = df['date'].iloc[-1]
        penultimate_date = df['date'].iloc[-2]
        if last_date > penultimate_date:
            step = +1
        else:
            step = -1

        df[new_metric] = df[metric].transform(lambda x: x.diff(periods=step)).clip(lower=0).fillna(0)
        
    return df

def unpack_list_of_dict(potential_list_of_dict, key_name, value_name):
    value = np.nan
    if type(potential_list_of_dict) is list:
        value = {entry.get(key_name):entry.get(value_name) for entry in potential_list_of_dict}
    return value
    
def df_create_categories(df, index_col, columns, category_field_name, value_field_name):
    df = df.filter(columns)
    df = df.melt(index_col, var_name=category_field_name, value_name=value_field_name)
    df = df[df[value_field_name].notnull()]
    df = df.groupby([index_col, category_field_name]).agg({value_field_name:'sum'})

    return df

In [ ]:
# load country information
# https://github.com/CSSEGISandData/COVID-19
def load_country_data():
    base_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data'
    file_name = f'UID_ISO_FIPS_LookUp_Table.csv'
    url = os.path.join(base_url, file_name)
    df = pd.read_csv(url)

    return df

def get_continent_from_iso2_code(iso2):
    try:
        continent = country_alpha2_to_continent_code(iso2)
    except:
        continent = 'Unknown'
        
    return continent

def add_continent(df):
    df_countries = load_country_data()
    df_countries = df_countries.filter(['Country_Region', 'Province_State', 'Combined_Key', 'iso2', 'iso3', 'Population'])

    # fix missing space in Johns Hopkins online data before merge
    combined_key_replace_dict = {'Northwest Territories,Canada': 'Northwest Territories, Canada'}
    df_countries['Combined_Key'] = df_countries['Combined_Key'].replace(combined_key_replace_dict)
    
    df = df.merge(df_countries, how='left', on='Combined_Key')
    df = df.drop_duplicates()

    df['continent'] = df['iso2'].apply(lambda x: get_continent_from_iso2_code(x))

    df.loc[df['iso2']=='SX', 'continent'] = 'NA' # add missing lookup for Sint Marten (Dutch part)
    df.loc[df['Combined_Key']=='Namibia', 'continent'] = 'AF' # add missing lookup for Namibia
    
    continent_dict = {'AN':'Antarctica', 'AS':'Asia', 'EU':'Europe', 'SA':'South America', 'NA':'North America', 'OC':'Oceania', 'AF':'Africa'}
    df['continent'] = df['continent'].replace(continent_dict)

    return df

# load data series
# - valid 'name' options are: 'confirmed', 'deaths', or 'recovered'
# - valid 'selection' options are: 'global' or 'US'
# https://github.com/CSSEGISandData/COVID-19
def load_individual_timeseries(name, selection='global', agg_states=True, keep_latlong=True):
    base_url = 'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series'
    file_name = f'time_series_covid19_{name}_{selection}.csv'
    
    if selection == 'global':
        index_col_list = ['Country/Region', 'Province/State', 'Lat', 'Long']
    else: # 'US'
        if name == 'confirmed':
            index_col_list = ['Country_Region', 'Province_State', 'Lat', 'Long_', 'UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Combined_Key']
        else: # 'deaths'
            index_col_list = ['Country_Region', 'Province_State', 'Lat', 'Long_', 'UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Admin2', 'Combined_Key', 'Population']

    url = os.path.join(base_url, file_name)
    df = pd.read_csv(url, index_col=index_col_list)
    df.columns.name = 'date'
    
    if not keep_latlong:
        df = df.reset_index(['Lat', 'Long'], drop=True)
        column_list = ['country', 'state', 'cases']
    else:
        if selection == 'US':
            if name == 'confirmed':
                columns_to_drop = ['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Combined_Key']
            else:
                columns_to_drop = ['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'Combined_Key', 'Population']

            df = df.reset_index(columns_to_drop, drop=True)

            column_list = ['country', 'state', 'lat', 'long', 'admin2', 'cases']

        else:
            column_list = ['country', 'state', 'lat', 'long', 'cases']

    df = df.stack()
    df = df.reset_index()
    df = df.set_index('date')
         
    df.index = pd.to_datetime(df.index)
    df.columns = column_list
    
    df['cases'] = df['cases'].astype(int)

    # Aggregate large countries split by states
    if agg_states:
        if selection == 'global':
            df_non_states = df[~df['country'].isin(countries_to_aggregate)]
            df_states     = df[ df['country'].isin(countries_to_aggregate)]
            df_states     = df_states.groupby(['country', 'date']).agg({'lat':'mean', 'long':'mean', 'cases':'sum'})
            df_states     = df_states.reset_index(['country'])
            df = pd.concat([df_non_states, df_states])
        else: # 'US'
            df = df.groupby(['country', 'state', 'date']).agg({'lat':'mean', 'long':'mean', 'cases':'sum'})
        
        df = df.reset_index()

    countries_to_drop = ['Diamond Princess'] # drop the cruise ship
    df = df[~df['country'].isin(countries_to_drop)]
    
    df['state'] = df['state'].fillna('')

    category = ['country', 'state']
#     df = df.rename(columns={'cases': 'cumCases'})
#     df = df_delta_from_cumulative(df, 'cumCases', category, 'cases')
#     df = df.drop(['cumCases'])

    return df

# - valid 'selection' options are: 'global' or 'US'
def load_covid_data(selection='global', agg_states=False, p_crit=.05, filter_n_days_100=None, keep_latlong=True):
    df = load_individual_timeseries('confirmed', selection=selection, agg_states=agg_states, keep_latlong=keep_latlong)
    df = df.rename(columns={'cases': 'cumConfirmed'})

    df['state'] = df['state'].fillna('')

    # Add deaths
    df_deaths = load_individual_timeseries('deaths', selection=selection)
    df_d = df_deaths.set_index(['country', 'state'], append=True)[['cases']]
    df_d.columns = ['cumDeaths']

    df = (df.set_index(['country', 'state'], append=True)
            .join(df_d)
            .reset_index(['country', 'state'])
    )
    
    df['Combined_Key'] = (df['state'] + ", " + df['country']).str.strip(', ')

    df = df.reset_index()

    df = add_continent(df) # consider to add continent for time series data too
    
    df = df_delta_from_cumulative(df, 'cumConfirmed', 'Combined_Key', 'confirmed')
    df = df_delta_from_cumulative(df, 'cumDeaths', 'Combined_Key', 'deaths')
    df = df.drop(['cumConfirmed', 'cumDeaths'], axis='columns')

    df['deathsRate']    = df['deaths']    / df['Population'] * 100E3 # per 100k population
    df['confirmedRate'] = df['confirmed'] / df['Population'] * 100E3 # per 100k population

    return df

# - valid 'selection' options are: 'global' or 'US'
def load_covid_data_us(agg_states=True, keep_latlong=True):
    selection = 'US'
    df = load_individual_timeseries('confirmed', selection=selection, agg_states=agg_states, keep_latlong=keep_latlong)
    df = df.rename(columns={'cases': 'cumConfirmed'})

    df['state'] = df['state'].fillna('')
    if not agg_states:
        df['admin2'] = df['admin2'].fillna('')
        index_col_list = ['country', 'state', 'admin2']
    else:
        index_col_list = ['country', 'state']
        
    # Add deaths
    df_deaths = load_individual_timeseries('deaths', selection=selection)
    df_d = df_deaths.set_index(index_col_list, append=True)[['cases']]
    df_d.columns = ['cumDeaths']

    df = (df.set_index(index_col_list, append=True)
            .join(df_d)
            .reset_index(index_col_list)
    )
    
    df = df.reset_index()

    df = df_delta_from_cumulative(df, 'cumConfirmed', 'state', 'confirmed')
    df = df_delta_from_cumulative(df, 'cumDeaths', 'state', 'deaths')
    df = df.drop(['cumConfirmed', 'cumDeaths'], axis='columns')
    
    df['Population'] = 0 # placeholder for now

    return df

def get_totals_from_cumulative_time_series(df):
    agg_dict = dict.fromkeys(df, 'last')
    agg_dict.update(dict.fromkeys(['cumConfirmed', 'cumDeaths'], 'last'))
    df_totals = df.copy().groupby(['country', 'state']).agg(agg_dict)
    df_totals = df_totals.rename(columns={'cumConfirmed':'confirmed', 'cumDeaths':'deaths'})
    df_totals = df_totals.reset_index(drop=True) # to ensure important fields are accessible in columns rather than hidden in index

    return df_totals

def get_totals_from_time_series(df):
    agg_dict = dict.fromkeys(df, 'last')
    agg_dict.update(dict.fromkeys(['confirmed', 'deaths'], 'sum'))
    df_totals = df.copy().groupby(['country', 'state']).agg(agg_dict)
    df_totals = df_totals.reset_index(drop=True) # to ensure important fields are accessible in columns rather than hidden in index

    return df_totals

def get_continent_totals(df):
    df_totals = df.copy().drop(['country', 'state'], axis=1)
    df_totals = df_totals.groupby(['continent']).agg({'lat':'mean', 'long':'mean', 'confirmed':'sum', 'deaths':'sum', 'Population':'sum'})
    df_totals = df_totals.reset_index() # to ensure important fields are accessible in columns rather than hidden in index
    
    df_totals['deathsRate']    = df_totals['deaths']    / df_totals['Population'] * 100E3 # per 100k population
    df_totals['confirmedRate'] = df_totals['confirmed'] / df_totals['Population'] * 100E3 # per 100k population
    
    return df_totals

In [ ]:
display_h1('COVID Data Visualisation')
display_h2('Load COVID data')

print('Loading World data')
df_covid_time_series_world = load_covid_data(agg_states=aggregate_states, filter_n_days_100=1, keep_latlong=True)
df_totals_world = get_totals_from_time_series(df_covid_time_series_world)

# optionally filter by num cases, to exclude what would otherwise be 'noisy' statistics
if num_cases_threshold:
    selected_list = df_totals_world.loc[df_totals_world['confirmed']>=num_cases_threshold, 'Combined_Key']
    exclude_list  = df_totals_world.loc[df_totals_world['confirmed']<num_cases_threshold, 'Combined_Key']
    print('- excluding cases <', num_cases_threshold)
    print('- excluded list:', list(exclude_list))
    df_totals_world = df_totals_world[df_totals_world['Combined_Key'].isin(selected_list)]
    df_covid_time_series_world = df_covid_time_series_world[df_covid_time_series_world['Combined_Key'].isin(selected_list)]
    
df_unknown_continents = df_covid_time_series_world[df_covid_time_series_world['continent']=='Unknown']['Combined_Key']
if len(df_unknown_continents):
    print('- unknown continents:', df_unknown_continents.unique())
    
print('')

print('Loading US data')
df_covid_time_series_us = load_covid_data_us()

print('')

print('Finished loading data')

print('')

latest_date = df_covid_time_series_world['date'].max()
latest_date_str = "{:%d-%b-%Y}".format(latest_date)
print('Most recent data: ', latest_date_str)

In [ ]:
# https://coronavirus-staging.data.gov.uk/details/about-data
# https://coronavirus-staging.data.gov.uk/details/interactive-map

def df_from_uk_gov(area_type = 'overview', area_name = None, latest=False):
    ENDPOINT = "https://api.coronavirus.data.gov.uk/v1/data"
    cases_field = 'cumCasesByPublishDate' if area_type == 'nation' else 'cumCasesBySpecimenDate'
    deaths_field = 'cumDeathsByDeathDate'

    # areaType: Area type as string
    # areaName: Area name as string
    # areaCode: Area Code as string
    # date: Date as string [YYYY-MM-DD]
    filters = [
        f"areaType={ area_type }" # overview, nation, region, nhsRegion, utla, ltla
    ]
    if area_name: filters = filters + [f"areaName={ area_name }"]

    # Some metrics are not available for specific areaType values.
    # For instance, we have newCasesByPublishDate and cumCasesByPublishDate only available for areaType=nation but not for region,utla, or ltla.
    # Conversely, we have newCasesBySpecimenDate and cumCasesBySpecimenDate available for region, utla, and ltla but not for nation.
    structure = {
        "date": "date",
        "type": "areaType",
        "name": "areaName",
        "code": "areaCode",
        "cases": cases_field,
        "deaths": deaths_field
    }

    api_params = {
        "filters": str.join(";", filters),
        "structure": dumps(structure, separators=(",", ":"))
    }
    if latest: api_params["latestBy"] = deaths_field # must be a metrics field included in 'structure'

    page_number = 1
    df_overall = pd.DataFrame() # initial value
    finished = False
    while not finished:
        #         print('- query page ', page_number)
        # Adding page number to query params
        api_params["page"] = page_number

        response = get(ENDPOINT, params=api_params, timeout=100)

        if response.status_code >= HTTPStatus.BAD_REQUEST:
            raise RuntimeError(f'Request failed: { response.text }')
        elif response.status_code == HTTPStatus.NO_CONTENT:
            print('No data returned from query')
            break

        df = pd.DataFrame() # initial value
        data = response.json()
        # {'length': 216, 'maxPageLimit': 1000, 'data': [{'date': '2020-08-07', 'name': 'England', 'code': 'E92000001', 'cases': {'daily': 796, 'cumulative': 266645}, 'deaths': {'daily': None, 'cumulative': None}}, {'date': '2020-08-06', 'name': 'England', 'code': 'E92000001', 'cases': {'daily': 826, 'cumulative': 265849}, 'deaths': {'daily': 14, 'cumulative': 41886}},
        df = pd.DataFrame.from_dict(data['data'])
        df_overall = pd.concat([df_overall, df])

        # check if more to read
        length = data['length']
        page_limit = data['maxPageLimit']
        if (length < page_limit) or (page_limit == 1):
            finished = True
        else:
            page_number += 1

    return df_overall
    
def df_from_uk_gov_v1(metrics_list, area_type='overview', area_code=None, timeout=200):
    url = 'https://api.coronavirus.data.gov.uk/v1/data'
    metrics_string = '&metric='.join(metrics_list)
    format = 'json'

    api_params = {}
    area_code = 'england'

#     filters = [
#         f"areaType={area_type}"
#     ]
#     api_params.filters = filters
#     if area_code is not None:
#         api_params.filters = [f'areaName={area_code}']
    api_params = 'filters=areaType=nation;areaName=england'
    api_params = api_params+'&structure={"date":"date","newCases":"newCasesByPublishDate"}'
    response = requests.get(url, params=api_params, timeout=timeout)

    df = pd.DataFrame()
    if response.status_code >= HTTPStatus.BAD_REQUEST:
        raise RuntimeError(f'Request failed: { response.text }')
    elif response.status_code == HTTPStatus.NO_CONTENT:
        print('-- No data returned from query')
    else: # ok
        data = response.json()
        display(data)
        df = pd.DataFrame.from_dict(data['body'])

    return df

def df_from_uk_gov_v2(metrics_list, area_type='overview', area_code=None, timeout=200):
    url = 'https://api.coronavirus.data.gov.uk/v2/data'
    metrics_string = '&metric='.join(metrics_list)
    format = 'json'

    api_params = 'areaType='+area_type
    if area_code is not None:
        api_params = api_params+'&'+'areaCode='+area_code
    api_params = api_params+'&metric='+metrics_string
    api_params = api_params+'&format='+format
    response = requests.get(url, params=api_params, timeout=timeout)

    df = pd.DataFrame()
    if response.status_code >= HTTPStatus.BAD_REQUEST:
        raise RuntimeError(f'Request failed: { response.text }')
    elif response.status_code == HTTPStatus.NO_CONTENT:
        print('-- No data returned from query')
    else: # ok
        data = response.json()

        df = pd.DataFrame.from_dict(data['body'])

    return df

def load_covid_data_uk_v1(metrics_dict, area_type='overview', area_code=None):
    metrics_list = [metric for metric, new_name in metrics_dict.items()]

    df = df_from_uk_gov_v1(metrics_list, area_type=area_type, area_code=area_code, timeout=200)
    if not df.empty:
        df = df.rename(columns=metrics_dict)
        df['date'] = pd.to_datetime(df['date'])

    return df

def load_covid_data_uk_v2(metrics_dict, area_type='overview', area_code=None):
    metrics_list = [metric for metric, new_name in metrics_dict.items()]

    df = pd.DataFrame()
    df = df_from_uk_gov_v2(metrics_list, area_type=area_type, area_code=area_code, timeout=200)
    if not df.empty:
        df = df.rename(columns=metrics_dict)
        df['date'] = pd.to_datetime(df['date'])

    return df

def df_from_json_file(url):
    data = urlopen(url).read()
    raw_json = json.loads(data)

    formatted_json = [feature['attributes'] for feature in raw_json['features']]
    df = pd.DataFrame(formatted_json)

    return df

def add_region_mapping_uk(df):
    df = df.copy()
    
    # Local Authority District to Region (April 2019) Lookup in England
    # https://geoportal.statistics.gov.uk/datasets/local-authority-district-to-region-april-2019-lookup-in-england/geoservice
    #  LAD19CD    LAD19NM    RGN19CD    RGN19NM    FID  
    url = 'https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/LAD19_RGN19_EN_LU/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json'
    columns_to_drop = ['FID']
    df_region_lookup = df_from_json_file(url)
    df_region_lookup = df_region_lookup.drop(columns_to_drop, axis=1)

    # Lower Tier Local Authority to Upper Tier Local Authority (April 2019) Lookup in England and Wales
    #  LTLA19CD    LTLA19NM    UTLA19CD    UTLA19NM    FID  
    url = 'https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/LTLA19_UTLA19_EW_LU/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json'
    df_authority_lookup = df_from_json_file(url)
    columns_to_drop = ['FID']
    df_authority_lookup = df_authority_lookup.drop(columns_to_drop, axis=1)
    df_authority_lookup = df_authority_lookup.drop_duplicates()
    
    local_authority_tiers_to_regions = df_authority_lookup.merge(df_region_lookup, how='left', left_on='LTLA19CD', right_on='LAD19CD')

    # Ward to Local Authority District to County to Region to Country (December 2019) Lookup in United Kingdom
    # https://geoportal.statistics.gov.uk/datasets/ward-to-local-authority-district-to-county-to-region-to-country-december-2019-lookup-in-united-kingdom/geoservice
    #  WD19CD    WD19NM    LAD19CD    LAD19NM    CTY19CD    CTY19NM  RGN19CD    RGN19NM    CTRY19CD    CTRY19NM    FID  
    url = 'https://services1.arcgis.com/ESMARspQHYMw9BZ9/arcgis/rest/services/WD19_LAD19_CTY19_OTH_UK_LU/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json'
    df_county_lookup = df_from_json_file(url)
    columns_to_filter = ['LAD19CD', 'LAD19NM', 'CTY19CD', 'CTY19NM', 'RGN19CD', 'RGN19NM', 'CTRY19CD', 'CTRY19NM']
    df_county_lookup = df_county_lookup.filter(columns_to_filter)
    df_county_lookup = df_county_lookup.drop_duplicates()

    columns_to_filter = ['RGN19CD', 'CTRY19CD', 'CTRY19NM']
    region_to_country = df_county_lookup.filter(columns_to_filter)
    region_to_country = region_to_country.drop_duplicates()
    local_authority_tiers_to_regions_to_countries = local_authority_tiers_to_regions.merge(region_to_country, how='left', on='RGN19CD')

    # Create dictionaries to map field values
    LTLA19CD_to_RGN19NM_dict = dict(set(zip(local_authority_tiers_to_regions_to_countries['LTLA19CD'], local_authority_tiers_to_regions_to_countries['RGN19NM'])))
    UTLA19CD_to_RGN19NM_dict = dict(set(zip(local_authority_tiers_to_regions_to_countries['UTLA19CD'], local_authority_tiers_to_regions_to_countries['RGN19NM'])))
    RGN19CD_to_CTRY19NM_dict = dict(set(zip(local_authority_tiers_to_regions_to_countries['RGN19CD'], local_authority_tiers_to_regions_to_countries['CTRY19NM'])))
    RGN19NM_to_CTRY19NM_dict = dict(set(zip(local_authority_tiers_to_regions_to_countries['RGN19NM'], local_authority_tiers_to_regions_to_countries['CTRY19NM'])))
    
    # Lookup location hierarchy
    df.loc[df['areaType']=='region', 'regionName'] = df['areaName']
    df.loc[df['areaType']=='utla', 'regionName'] = df['areaCode'].map(UTLA19CD_to_RGN19NM_dict)
    df.loc[df['areaType']=='ltla', 'regionName'] = df['areaCode'].map(LTLA19CD_to_RGN19NM_dict)
    df['CountryName'] = df['regionName'].map(RGN19NM_to_CTRY19NM_dict)
    df.loc[df['areaType']=='nation', 'countryName'] = df['areaName']

    return df

def add_msoa_region_mapping_uk(df):
    df = df.copy()

    uk_msoa_lookup_file = 'uk_msoa_lookup_table.csv'
    uk_msoa_lookup_table = pd.read_csv(uk_msoa_lookup_file)

    MSOA11CD_to_LTLA_dict = dict(set(zip(uk_msoa_lookup_table['MSOA'], uk_msoa_lookup_table['LTLA_areaName'])))
    MSOA11CD_to_RGN_dict = dict(set(zip(uk_msoa_lookup_table['MSOA'], uk_msoa_lookup_table['region_areaName'])))

    df['LTLAName']   = df['areaCode'].map(MSOA11CD_to_LTLA_dict)
    df['RegionName'] = df['areaCode'].map(MSOA11CD_to_RGN_dict)

    return df

# get UK population for specified area codes
def df_get_uk_population():
    # UK population - mid 2019
    # Could not find a useable online source, so downloaded and extracted to CSV file
    # https://www.ons.gov.uk/file?uri=%2fpeoplepopulationandcommunity%2fpopulationandmigration%2fpopulationestimates%2fdatasets%2fpopulationestimatesforukenglandandwalesscotlandandnorthernireland%2fmid2019april2020localauthoritydistrictcodes/ukmidyearestimates20192020ladcodes.xls
    # Extracted subset of data to local CSV file
    uk_population_file_2019 = 'UK_Population_Mid_2019.csv'
    df_uk_population_2019 = pd.read_csv(uk_population_file_2019, thousands=',')

    return df_uk_population_2019

# Load UK Metrics
## Area type
area_overview = 'overview'
area_nation   = 'nation'
area_region   = 'region'
area_utla     = 'utla'
area_ltla     = 'ltla'
area_msoa     = 'msoa'

## Area code
area_code_all = None
area_code_England = 'E92000001'

## Metrics
overall_metrics_dict  = {'newCasesByPublishDate':'confirmed', 'newDeaths28DaysByDeathDate':'deaths',
                         'newCasesByPublishDateRollingRate':'confirmedRate', 'newDeaths28DaysByDeathDateRollingRate':'deathsRate'}
region_metrics_dict   = {'newCasesBySpecimenDate':'confirmed', 'newDeaths28DaysByDeathDate':'deaths',
                         'newCasesBySpecimenDateRollingRate':'confirmedRate', 'newDeaths28DaysByDeathDateRate':'deathsRate'}
hospital_metrics_dict = {'newAdmissions':'hospitalAdmissions', 'covidOccupiedMVBeds':'covidOccupiedMVBeds', 'hospitalCases':'hospitalCases'}
hospital_age_metrics_dict = {'cumAdmissionsByAge':'cumAdmissionsByAge'}
#vaccine_metrics_dict  = {'newPeopleReceivingFirstDose':'vaccineFirstDose', 'newPeopleReceivingSecondDose':'vaccineSecondDose'}
vaccine_metrics_dict  = {'cumPeopleVaccinatedFirstDoseByPublishDate':'cumPeopleVaccinatedFirstDoseByPublishDate', 
                         'cumPeopleVaccinatedFirstDoseByVaccinationDate':'cumPeopleVaccinatedFirstDoseByVaccinationDate',
                         'cumPeopleVaccinatedSecondDoseByPublishDate':'cumPeopleVaccinatedSecondDoseByPublishDate',
                         'cumPeopleVaccinatedSecondDoseByVaccinationDate':'cumPeopleVaccinatedSecondDoseByVaccinationDate'
                        }
# Nation & UK overview data:
## older data using: cumPeopleVaccinatedFirstDoseByVaccinationDate
## looks like might have moved on 11-Jan to: cumPeopleVaccinatedFirstDoseByPublishDate
## https://api.coronavirus.data.gov.uk/v2/data?areaType=overview&metric=newPeopleVaccinatedFirstDoseByPublishDate&format=csv
test_metrics_dict     = {'uniquePeopleTestedBySpecimenDateRollingSum':'peopleTested'}
msoa_metrics_dict     = {'newCasesBySpecimenDateRollingRate':'value'}
cases_age_metrics_dict= {'newCasesBySpecimenDateAgeDemographics':'confirmedAge'}
deaths_age_metrics_dict= {'newDeaths28DaysByDeathDateAgeDemographics':'deathsAge'}

In [ ]:
print('Loading UK data:')
print('- overview')
df_uk_overview = load_covid_data_uk_v2(metrics_dict=overall_metrics_dict, area_type=area_nation, area_code=area_code_all)
print('- hospital')
df_uk_hospital_england = load_covid_data_uk_v2(metrics_dict=hospital_metrics_dict, area_type=area_nation, area_code=area_code_England)
df_uk_hospital_age_england = load_covid_data_uk_v2(metrics_dict=hospital_age_metrics_dict, area_type=area_nation, area_code=area_code_England)
print('- vaccinations')
df_uk_vaccinations = load_covid_data_uk_v2(metrics_dict=vaccine_metrics_dict, area_type=area_nation, area_code=area_code_all)
if not df_uk_vaccinations.empty:
    df_uk_vaccinations['cumVaccineFirstDose'] = df_uk_vaccinations['cumPeopleVaccinatedFirstDoseByPublishDate'].fillna(
                                                    df_uk_vaccinations['cumPeopleVaccinatedFirstDoseByVaccinationDate'])
    df_uk_vaccinations['cumVaccineSecondDose'] = df_uk_vaccinations['cumPeopleVaccinatedSecondDoseByPublishDate'].fillna(
                                                    df_uk_vaccinations['cumPeopleVaccinatedSecondDoseByVaccinationDate'])
    df_uk_vaccinations = df_uk_vaccinations.drop(['cumPeopleVaccinatedFirstDoseByVaccinationDate',  'cumPeopleVaccinatedFirstDoseByPublishDate',
                                                  'cumPeopleVaccinatedSecondDoseByVaccinationDate', 'cumPeopleVaccinatedSecondDoseByPublishDate'],
                                                 axis='columns', errors='ignore')
print('- LTLA')
df_uk_ltla = load_covid_data_uk_v2(metrics_dict=region_metrics_dict, area_type=area_ltla, area_code=area_code_all)
df_uk_ltla_regions = add_region_mapping_uk(df_uk_ltla)
print('- MSOA')
df_uk_msoa = load_covid_data_uk_v2(metrics_dict=msoa_metrics_dict, area_type=area_msoa, area_code=area_code_all)
df_uk_msoa_with_regions = add_msoa_region_mapping_uk(df_uk_msoa)

print('- overall cases by age')
df_uk_age = load_covid_data_uk_v2(metrics_dict=cases_age_metrics_dict, area_type=area_nation, area_code=area_code_all)
df_deaths_uk_age = load_covid_data_uk_v2(metrics_dict=deaths_age_metrics_dict, area_type=area_nation, area_code=area_code_all)
if not df_uk_age.empty:
    df_uk_age['casesByAge'] = df_uk_age['confirmedAge'].apply(lambda x : unpack_list_of_dict(x, 'age', 'cases'))
    df_uk_age['casesRateByAge'] = df_uk_age['confirmedAge'].apply(lambda x : unpack_list_of_dict(x, 'age', 'rollingRate'))
    df_uk_age['deathsByAge'] = df_deaths_uk_age['deathsAge'].apply(lambda x : unpack_list_of_dict(x, 'age', 'deaths'))
    df_uk_age['deathsRateByAge'] = df_deaths_uk_age['deathsAge'].apply(lambda x : unpack_list_of_dict(x, 'age', 'rollingRate'))
    df_uk_age = df_uk_age.drop(['confirmedAge', 'deathsAge'], axis='columns', errors='ignore')

print('- UK population 2019')
df_uk_population_2019 = df_get_uk_population()

#population_uk = df_uk_population_2019.loc[df_uk_population_2019['Area name']=='UNITED KINGDOM', 'Population'].values[0]
population_uk = 66796807
population_great_britain = 64903140
population_england_and_wales = 59439840
population_england = 56286961
population_scotland = population_great_britain - population_england_and_wales
population_wales = population_england_and_wales - population_england
population_northern_ireland = population_uk - population_great_britain

# print('UK', population_uk)
# print('England', population_england)
# print('Scotland', population_scotland)
# print('Wales', population_wales)
# print('Northern Ireland', population_northern_ireland)

df_uk_vaccinations.loc[df_uk_vaccinations['areaName']=='England', 'Population'] = population_england
df_uk_vaccinations.loc[df_uk_vaccinations['areaName']=='Scotland', 'Population'] = population_scotland
df_uk_vaccinations.loc[df_uk_vaccinations['areaName']=='Wales', 'Population'] = population_wales
df_uk_vaccinations.loc[df_uk_vaccinations['areaName']=='Northern Ireland', 'Population'] =  population_northern_ireland

print('Finished loading UK data')

In [ ]:
print('Loading World-wide vaccination data')

def get_vaccine_trend_from_field_list(df, data_field, field):
    df = df.copy()
    
    index_cols = df.columns
    df[field] = df[data_field].apply(lambda x : unpack_list_of_dict(x, 'date', field))

    df_field_list = pd.DataFrame(df[field].to_list())
    df = pd.concat([df[index_cols], df_field_list], axis='columns')

    df = df.melt(index_cols, var_name='date', value_name=field)

    df = df.drop([data_field], axis='columns')

    return df
    
def get_country_population():
    url = 'https://github.com/owid/covid-19-data/blob/master/scripts/input/un/population_2020.csv?raw=true'
    df = pd.read_csv(url)
    
    return df

def df_from_world_in_data_csv():
    url = 'https://github.com/owid/covid-19-data/blob/master/public/data/vaccinations/vaccinations.csv?raw=true'

    df = pd.read_csv(
        url, 
        parse_dates=['date'])

    df = df.sort_values(by=['date'], ascending=True)

    iso_codes_to_drop = df[df['iso_code'].str.startswith('OWID_')]['iso_code'].unique()
    df = df[~df['iso_code'].str.startswith('OWID_')]

    df['iso2'] = df['iso_code'].apply(lambda x: pycountry_convert.country_alpha3_to_country_alpha2(x))
    df['continent'] = df['iso2'].apply(lambda x: get_continent_from_iso2_code(x))
    df = df.drop(['iso2', 'iso_code'], axis='columns')

    continent_dict = {'AN':'Antarctica', 'AS':'Asia', 'EU':'Europe', 'SA':'South America', 'NA':'North America', 'OC':'Oceania', 'AF':'Africa'}
    df['continent'] = df['continent'].replace(continent_dict)
    df = df.rename(columns={'location':'country'})

    df_pop = get_country_population()
    
    country_population_dict = dict(set(zip(df_pop['entity'], df_pop['population'])))
    df['population'] = df['country'].map(country_population_dict)

    return df

df_world_vaccines = df_from_world_in_data_csv()
# display(df_world_vaccines.head())

In [ ]:
__version__ = "0.1.00"

import os # For file path handling  

# to obtain the pptx module:
#   pip install --user python-pptx
import pptx
from pptx.util import Mm
from pptx.util import Length


class PPT():
    OUTPUT_FILE_NAME = 'test_slides.pptx'
    IMAGE_FILE_NAME = 'test_chart.png'
    DIRECTORY_NAME = '.' # option to put in sub-directory
    
    # folder structure
    generated_chart = os.path.join(DIRECTORY_NAME, IMAGE_FILE_NAME)
    generated_ppt = os.path.join(DIRECTORY_NAME, OUTPUT_FILE_NAME)
    
    # slide template enumerations
    from enum import IntEnum
    class slide_template_layout(IntEnum):
        TITLE = 0
        TITLE_AND_CONTENT = 1
        TITLE_AND_MAX_CONTENT = 2
        
    placeholders = {
        slide_template_layout.TITLE:{'title':{'id':0, 'type':'text'}, 'subtitle':{'id':1, 'type':'text'}},
        slide_template_layout.TITLE_AND_CONTENT:{'title':{'id':0, 'type':'text'}, 'content':{'id':1, 'type':'content'}, 'footer':{'id':10, 'type':'text'}},
        slide_template_layout.TITLE_AND_MAX_CONTENT:{'title':{'id':0, 'type':'text'}, 'content':{'id':1, 'type':'content'}, 'footer':{'id':10, 'type':'text'}},
    }
    
    def __init__(self, template=None, outfile=None, placeholders=None, slide_width_mm=None, slide_height_mm=None):
        self.template = template
        self.outfile = outfile if outfile else self.generated_ppt
        self.placeholders = placeholders if placeholders else self.placeholders
        
        if template:
            print('Opening slides:', self.template)

        self.ppt = pptx.Presentation(self.template)
        self.slides = self.ppt.slides
        
        if (not slide_width_mm is None) and (not slide_height_mm is None):
            self.ppt.slide_width = Mm(slide_width_mm)
            self.ppt.slide_height = Mm(slide_height_mm)
        
    # Add slide
    def add_slide(self, layout):
        slide_layout = self.ppt.slide_layouts[layout]
        slide = self.slides.add_slide(slide_layout)
        return slide
    
    def set_text(self, slide, placeholder_name, text):
        try:
            layout = self.ppt.slide_layouts.index(slide.slide_layout)
            placeholder_id = self.placeholders.get(layout,{}).get(placeholder_name,{}).get('id',None)
            placeholder = slide.placeholders[placeholder_id]
            placeholder.text = text
        except Exception as e:
            print('Failed to add text to placeholder: ', placeholder_name)
            print(e)
            pass
        
    def add_textbox(self, slide, text, left_mm, top_mm, width_mm, height_mm):
        txtBox = slide.shapes.add_textbox(Mm(left_mm), Mm(top_mm), Mm(width_mm), Mm(height_mm))
        txtBox.text_frame.text = text
    
    def get_slide_placeholder_size(self, slide, placeholder_name):
        try:
            layout = self.ppt.slide_layouts.index(slide.slide_layout)
            placeholder_id = self.placeholders.get(layout,{}).get(placeholder_name,{}).get('id',None)
            placeholder = slide.placeholders[placeholder_id]
            sizes = (placeholder.left.mm, placeholder.top.mm, placeholder.width.mm, placeholder.height.mm)
        except Exception as e:
            print('Failed to get placeholder: ', placeholder_name)
            print(e)
            pass
        return sizes
    
    def get_slide_size(self):
        slide_width = self.ppt.slide_width
        slide_height = self.ppt.slide_height
        return (Length(slide_width).mm, Length(slide_height).mm)

    def add_image(self, slide, placeholder_name, image_file, keep_aspect_ratio=True, position='centre', delete_placeholder=True):
        try:
            # read image file to check actual aspect ratio
            img = plt.imread(image_file)
            img_height, img_width = img.shape[0], img.shape[1]
            image_aspect_ratio = img_width/img_height
            
            # get placeholder details, including aspect ratio
            layout = self.ppt.slide_layouts.index(slide.slide_layout)
            placeholder_id = self.placeholders.get(layout,{}).get(placeholder_name,{}).get('id',None)
            placeholder = slide.placeholders[placeholder_id]
            placeholder_aspect_ratio = placeholder.width/placeholder.height
            
            # default positions; later calculate offset for centering if appropriate
            x_pos = placeholder.left
            y_pos = placeholder.top
            width = placeholder.width
            height = placeholder.height
            
            # determine which dimensions to use
            if keep_aspect_ratio: # only set 1 dimension of image, to keep aspect ratio
                use_image_width=True
                if image_aspect_ratio < placeholder_aspect_ratio:
                    use_image_width=False

                if use_image_width:
                    if position=='centre':
                        new_img_height = img_height/img_width*width
                        y_pos = y_pos + (height - new_img_height)/2
                    pic = slide.shapes.add_picture(image_file, x_pos, y_pos, width=width)
                else:
                    if position=='centre':
                        new_img_width = img_width/img_height*height
                        x_pos = x_pos + (width - new_img_width)/2
                    pic = slide.shapes.add_picture(image_file, x_pos, y_pos, height=height)
                    
            else: # set both dimensions of image, which can therefore change aspect ratio
                pic = slide.shapes.add_picture(image_file, x_pos, y_pos, width=width, height=height)
                
            if delete_placeholder:
                sp = placeholder._sp # get shape xml element
                sp.getparent().remove(sp) # remove shape

        except Exception as e:
            print('Failed to add image to placeholder: ', placeholder_name, ', image filename:', image_file)
            print(e)
            pass

    def save(self, outfile=None):
        outfile = outfile if outfile else self.outfile
        if not outfile:
            print('Need to specify filename to save; existing')
            quit()
        self.ppt.save(outfile)

    def slide_describe(self, slide, layout_name):
        print('Slide layout:', layout_name)
        try:
            set_title(slide, layout_name)
        except:
            pass

        txBox = slide.shapes.add_textbox(0, 0, 100, 100)
        tf = txBox.text_frame
        tf.text = f'{layout_name}; Placeholders:'
        for placeholder in slide.placeholders:
            size_description = ("left=%d, top=%d, width=%d, height=%d (mm)" % (placeholder.left.mm, placeholder.top.mm, placeholder.width.mm, placeholder.height.mm))
            num_name = '%d; %s' % (placeholder.placeholder_format.idx, placeholder.name)
            name_name_size = num_name + " : " + size_description

            p = tf.add_paragraph()
            p.text = name_name_size

    def new_slide_describe(self, layout):
        slide = self.add_slide(layout)
        try:
            layout_name = self.ppt.slide_layouts[layout].name #layout.name
        except:
            layout_name = ''
        self.slide_describe(slide, layout_name)

    def add_describe_template(self, known_layouts_only = True):
        # to do: derive slide list to populate from template
        if not known_layouts_only:
            for slide_layout_num, slide_layout in enumerate(self.ppt.slide_layouts):
                self.new_slide_describe(slide_layout_num)
        else:
            self.new_slide_describe(self.slide_template_layout.TITLE)
            self.new_slide_describe(self.slide_template_layout.TITLE_AND_CONTENT)
            self.new_slide_describe(self.slide_template_layout.TITLE_AND_MAX_CONTENT)

In [ ]:
annotate_template = False
if annotate_template:
    print('Describe slide template')
    prs = PPT(template_pptx) # create new presentation using default template
    prs.add_describe_template(known_layouts_only=False)
    prs.save('template_annotated.pptx')
    
if export_slides:
    # Create new presentation from (empty) template
    prs = PPT(template_pptx) # create new presentation using default template

In [ ]:
slide_title = 'COVID-19 Summary'
display_h1(slide_title)
text_str = 'Analysis run: '+ run_date_str + '\n'
text_str = text_str + 'Latest data: '+ latest_date_str + '\n'
text_str = text_str + 'Total confirmed cases: {:,d}'.format(df_totals_world['confirmed'].sum().astype(int)) + '\n'
text_str = text_str + 'Total deaths: {:,d}'.format(df_totals_world['deaths'].sum().astype(int)) + '\n'
text_str = text_str + '\n'
text_str = text_str + 'COVID-19 data: https://github.com/CSSEGISandData/COVID-19' + '\n'
text_str = text_str + 'Mortality data: https://www.mortality.org' + '\n'
text_str = text_str + 'UK data: https://coronavirus.data.gov.uk' +'\n'
text_str = text_str + 'Vaccine data: https://ourworldindata.org/covid-vaccinations' + '\n'
text_str = text_str + '\n'

text_str = text_str + 'Analysis & summary by Colin Osborne: https://github.com/colin0brass/covid' + '\n'
# text_str = text_str + 'Country/region included in trends if confirmed cases above: {:,d}'.format(num_cases_threshold)

print(text_str)

if export_slides:
    # Add a new slide ready for chart and set title text
    slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_CONTENT)
    prs.set_text(slide, placeholder_name='title', text=slide_title)
    prs.set_text(slide, placeholder_name='content', text=text_str)

In [ ]:
def trend_chart(df, metric='deaths', category=None, area_chart=True, plot_7_day_average=True,
                x_min=None, x_max=None, y_min=None, y_max=None, y_range_from_avg=True, filter_last_weeks=None,
                forward_fill=False, remove_trailing_zeros=True, per_population_percent=False,
                chart_width=350, chart_height=150,
                row_title='', column_title=''):
    df = df.copy()

    index_col_list = ['date']
    if category:
        category_list=['date', category]
        
        if forward_fill:
            df[metric] = df.groupby([category], sort=False)[metric].apply(lambda x: x.ffill())

        if per_population_percent:
            df = df.groupby(category_list).agg({metric:'sum', 'Population':'sum'})
            df = df.reset_index()
            df[metric] = df[metric] / df['Population'] * 100
        else:
            df = df.groupby(category_list).agg({metric:'sum'})
            df = df.reset_index()

    if remove_trailing_zeros:
        df = df_remove_trailing_zeros(df, metric)
        
    if plot_7_day_average:
        line_metric = 'weekAvg'
        if category:
            df[line_metric] = df.groupby(category)[metric].rolling(window=7, min_periods=7).mean().reset_index(0, drop=True)
        else:
            df[line_metric] = df[metric].rolling(window=7, min_periods=7).mean().reset_index(0, drop=True)
    else:
        line_metric = metric

    if y_range_from_avg:
        metric_for_y_range = line_metric
    else:
        metric_for_y_range = metric
        
    x_min = df['date'].min() if not x_min else x_min
    x_max = df['date'].max() if not x_max else x_max
    
    if filter_last_weeks:
        x_min = x_max - datetime.timedelta(weeks=filter_last_weeks)
        df = df[df['date']>=x_min]

    if y_max is None: y_max = df[metric_for_y_range].max()
    if y_min is None: y_min = df[metric_for_y_range].min()

    chart = alt.Chart()
    if area_chart:
        chart = alt.Chart().mark_area().encode(
            x=alt.X('date:T', scale=alt.Scale(domain=[x_min, x_max])),
            y   = metric+':Q',
            tooltip=[alt.Tooltip('date:T'),
                     alt.Tooltip(line_metric+':Q', format=',d'),
                     alt.Tooltip(metric+':Q', format=',d')]
        )

        line = alt.Chart().mark_line(
            color='red',
        ).encode(
            x=alt.X('date:T', scale=alt.Scale(domain=[x_min, x_max])),
            y=alt.Y(line_metric+':Q', title=row_title, scale=alt.Scale(domain=[y_min, y_max], clamp=True)),
            tooltip=[alt.Tooltip('date:T'),
                     alt.Tooltip(line_metric+':Q', format=',d'),
                     alt.Tooltip(metric+':Q', format=',d')]
        )
        
        chart = alt.layer(chart, line, data=df)

    else: # not area chart
        line = alt.Chart().mark_line(
        ).encode(
            x=alt.X('date:T', scale=alt.Scale(domain=[x_min, x_max])),
            y=alt.Y(line_metric+':Q', title=row_title, scale=alt.Scale(domain=[y_min, y_max], clamp=True)),
            color=alt.Color(category+':N', sort=None),
            tooltip=[alt.Tooltip('date:T'),
                     alt.Tooltip(category+':N'),
                     alt.Tooltip(line_metric+':Q', format=',d')]
        )
        
        chart = alt.layer(line, data=df)
        
    chart = chart.properties(title=column_title, height=chart_height, width=chart_width)

    return chart

def bar_chart(df, metric='confirmed', sort_field='deaths', numbers_overlaid=False, category=None, per_population_percent=False, 
              x_min=None, x_max=None, chart_width=200, chart_height=150, rollup='latest', forward_fill=False, row_title='', column_title=''):
    df = df.copy()
    
    if category:
        category_list=['date', category]
        
        if forward_fill:
            df[metric] = df.groupby([category], sort=False)[metric].apply(lambda x: x.ffill())

        if per_population_percent:
            if sort_field in df.columns:
                df = df.groupby(category_list).agg({metric:'sum', sort_field:'sum', 'Population':'sum'})
            else:
                df = df.groupby(category_list).agg({metric:'sum', 'Population':'sum'})
            df = df.reset_index()
            df[metric] = df[metric] / df['Population'] * 100
        else:
            if sort_field in df.columns:
                df = df.groupby(category_list).agg({metric:'sum', sort_field:'sum'})
            else:
                df = df.groupby(category_list).agg({metric:'sum'})
            df = df.reset_index()
        
    if rollup=='latest':
#         df = df[df['date']==df['date'].max()] # last value
        df = df.groupby(category).last() # last
        df = df.reset_index()
    elif rollup=='total':
        df = df.groupby(category).sum() # total
        df = df.reset_index()
    else:
        print('Unknown rollup option:', rollup)
        
    x_min = 0 # df[metric].min() if not x_min else x_min
    x_max = df[metric].max() if not x_max else x_max

    sort_order = None # initial value
    if sort_field and sort_field in df.columns and category is not None:
        sort_order = list(df.sort_values(by=sort_field, ascending=False)[category].unique())

    if numbers_overlaid:
        bars = alt.Chart(df).mark_bar(
        ).encode(
            x=alt.X(metric+':Q', scale=alt.Scale(domain=[x_min, x_max])),
            y=alt.Y(category+':N', sort=sort_order, title=row_title), # sort='-x' to sort locally in this chart, or 'None' for no sort
    #         color=alt.Color(f'{category}:N', sort=None, legend=None), # removed since this overrode the text colour below
            tooltip=[alt.Tooltip(category+':N'),
                     alt.Tooltip(metric+':Q', format=',d')]
        )

        text = bars.mark_text(
            align = 'right',
            baseline = 'middle',
            dx = -15,
            color = 'white',
            fontSize=20
        ).encode(
            text=alt.Text(f'{metric}:Q', format=',d') #,.2r gives rounded to 2 significant figures
        )
    else:
        bars = alt.Chart(df).mark_bar(
        ).encode(
            x=alt.X(metric+':Q', scale=alt.Scale(domain=[x_min, x_max])),
            y=alt.Y(category+':N', sort=sort_order, title=row_title), # sort='-x' to sort locally in this chart, or 'None' for no sort
            color=alt.Color(category+':N', sort=None, legend=None, scale=alt.Scale(scheme='category20')),
            tooltip=[alt.Tooltip(category+':N'),
                     alt.Tooltip(f'{metric}:Q', format=',d')]
        )

        text = bars.mark_text(
            align    = 'left',
            baseline = 'middle',
            dx       = 3 # Nudges text to right so it doesn't appear on top of the bar
        ).encode(
            text=alt.Text(f'{metric}:Q', format=',d') #,.2r gives rounded to 2 significant figures
        )
    
    chart = (bars + text)
    chart = chart.properties(title=column_title, height=chart_height, width=chart_width)
    
    return chart

def configure_trend_chart(chart):
    chart = chart.configure_axisY(
        titleColor='blue',
        titleFontSize=20,
    )
    
    chart = chart.configure_title(
        fontSize=20,
        anchor='middle',
        color='blue'
    )
    
    return chart

In [ ]:
slide_title = 'Covid-19 Worldwide'
display_h1(slide_title)

def df_world_time_series_summary(df):
    df_totals = df.copy().drop(['continent', 'Combined_Key', 'country', 'state'], axis=1)
    df_totals = df_totals.groupby(['date']).agg({'lat':'mean', 'long':'mean', 
                                                 'confirmed':'sum', 'deaths':'sum', 
                                                 'Population':'sum'})
    df_totals['confirmedRate'] = df_totals['confirmed'] / df_totals['Population'] * 100E3 # per 100k population
    df_totals['deathsRate']    = df_totals['deaths']    / df_totals['Population'] * 100E3 # per 100k population
    df_totals['areaType'] = 'World'
    df_totals = df_totals.reset_index()

    return df_totals

def df_continents_time_series_summary(df):
    df_totals = df.copy().drop(['Combined_Key', 'country', 'state'], axis=1)
    df_totals = df_totals.groupby(['date', 'continent']).agg({'lat':'mean', 'long':'mean', 
                                                              'confirmed':'sum', 'deaths':'sum', 
                                                              'Population':'sum'})

    df_totals['confirmedRate'] = df_totals['confirmed'] / df_totals['Population'] * 100E3 # per 100k population
    df_totals['deathsRate']    = df_totals['deaths']    / df_totals['Population'] * 100E3 # per 100k population

    df_totals = df_totals.reset_index()

    return df_totals

def get_world_overview_chart(df):
    df = df.copy()
    
    chart_width_large = 300
    chart_width_small = 150
    chart_width_smaller = 110
    chart_height = 150
    
    df_totals = get_totals_from_time_series(df)

    df_continent_totals_absolute       = get_continent_totals(df_totals)

    df_world_time_series = df_world_time_series_summary(df)
    df_continents_time_series = df_continents_time_series_summary(df)

    x_min = df['date'].min()
    x_max = df['date'].max()
    
    trend_confirmed_cases  = trend_chart(df_world_time_series, metric='confirmed', category=None, x_min=x_min, x_max=x_max, 
                                         chart_width=chart_width_large, chart_height=chart_height, 
                                         row_title='World', column_title='Daily Cases')
    trend_confirmed_cases_recent  = trend_chart(df_world_time_series, metric='confirmed', category=None, x_min=x_min, x_max=x_max, filter_last_weeks=4,
                                         chart_width=chart_width_smaller, chart_height=chart_height, 
                                         row_title='', column_title='Cases 4wks')
    trend_confirmed_deaths = trend_chart(df_world_time_series, metric='deaths',    category=None, x_min=x_min, x_max=x_max, 
                                         chart_width=chart_width_large, chart_height=chart_height, 
                                         row_title='', column_title='Daily Deaths')
    trend_confirmed_deaths_recent = trend_chart(df_world_time_series, metric='deaths',    category=None, x_min=x_min, x_max=x_max,  filter_last_weeks=4,
                                         chart_width=chart_width_smaller, chart_height=chart_height, 
                                         row_title='', column_title='Deaths 4wks')
    total_confirmed_cases  = bar_chart(df_world_time_series, metric='confirmed', category='areaType', rollup='total', numbers_overlaid=True,
                                         chart_width=chart_width_small, chart_height=chart_height, 
                                       row_title='', column_title='Total Cases')
    total_confirmed_deaths  = bar_chart(df_world_time_series, metric='deaths',   category='areaType', rollup='total', numbers_overlaid=True,
                                         chart_width=chart_width_small, chart_height=chart_height, 
                                        row_title='', column_title='Total Deaths')
    trend_row = trend_confirmed_cases | trend_confirmed_cases_recent | trend_confirmed_deaths | trend_confirmed_deaths_recent | total_confirmed_cases | total_confirmed_deaths
    
    trend_confirmed_cases   = trend_chart(df_continents_time_series, metric='confirmed', category='continent', x_min=x_min, x_max=x_max, 
                                         chart_width=chart_width_large, chart_height=chart_height, 
                                          area_chart=False, row_title='Continents', column_title='Daily Cases')
    trend_confirmed_cases_recent   = trend_chart(df_continents_time_series, metric='confirmed', category='continent', x_min=x_min, x_max=x_max, filter_last_weeks=4,
                                         chart_width=chart_width_smaller, chart_height=chart_height, 
                                          area_chart=False, row_title='', column_title='Cases 4wks')
    trend_confirmed_deaths  = trend_chart(df_continents_time_series, metric='deaths',    category='continent', x_min=x_min, x_max=x_max, 
                                          remove_trailing_zeros=True,
                                         chart_width=chart_width_large, chart_height=chart_height, 
                                          area_chart=False, row_title='', column_title='Daily Deaths')
    trend_confirmed_deaths_recent  = trend_chart(df_continents_time_series, metric='deaths',    category='continent', x_min=x_min, x_max=x_max, filter_last_weeks=4,
                                          remove_trailing_zeros=True,
                                         chart_width=chart_width_smaller, chart_height=chart_height, 
                                          area_chart=False, row_title='', column_title='Deaths 4wks')
    total_confirmed_cases  = bar_chart(df_continents_time_series, metric='confirmed', category='continent', rollup='total', numbers_overlaid=False,
                                         chart_width=chart_width_small, chart_height=chart_height, 
                                       row_title='', column_title='Total Cases')
    total_confirmed_deaths  = bar_chart(df_continents_time_series, metric='deaths',   category='continent', rollup='total', numbers_overlaid=False,
                                         chart_width=chart_width_small, chart_height=chart_height, 
                                        row_title='', column_title='Total Deaths')
    continents_row = trend_confirmed_cases | trend_confirmed_cases_recent | trend_confirmed_deaths | trend_confirmed_deaths_recent | total_confirmed_cases | total_confirmed_deaths
    
    trend_confirmed_cases   = trend_chart(df_continents_time_series, metric='confirmedRate', category='continent', x_min=x_min, x_max=x_max, 
                                          area_chart=False, remove_trailing_zeros=True, plot_7_day_average=True,
                                         chart_width=chart_width_large, chart_height=chart_height, 
                                          row_title='Continents', column_title='Daily Cases per 100k')
    trend_confirmed_cases_recent   = trend_chart(df_continents_time_series, metric='confirmedRate', category='continent', x_min=x_min, x_max=x_max, filter_last_weeks=4,
                                          area_chart=False, remove_trailing_zeros=True, plot_7_day_average=True,
                                         chart_width=chart_width_smaller, chart_height=chart_height, 
                                          row_title='', column_title='Cases 4wks/100k')
    trend_confirmed_deaths  = trend_chart(df_continents_time_series, metric='deathsRate',    category='continent', x_min=x_min, x_max=x_max, 
                                          area_chart=False, remove_trailing_zeros=True, plot_7_day_average=True,
                                         chart_width=chart_width_large, chart_height=chart_height, 
                                          row_title='', column_title='Daily Deaths per 100k')
    trend_confirmed_deaths_recent  = trend_chart(df_continents_time_series, metric='deathsRate',    category='continent', x_min=x_min, x_max=x_max, filter_last_weeks=4,
                                          area_chart=False, remove_trailing_zeros=True, plot_7_day_average=True,
                                         chart_width=chart_width_smaller, chart_height=chart_height, 
                                          row_title='', column_title='Deaths 4wks/100k')
    total_confirmed_cases  = bar_chart(df_continents_time_series, metric='confirmedRate', sort_field='deathsRate', category='continent', rollup='total', numbers_overlaid=False,
                                         chart_width=chart_width_small, chart_height=chart_height, 
                                       row_title='', column_title='Total Cases per 100k')
    total_confirmed_deaths  = bar_chart(df_continents_time_series, metric='deathsRate',   sort_field='deathsRate', category='continent', rollup='total', numbers_overlaid=False,
                                         chart_width=chart_width_small, chart_height=chart_height, 
                                        row_title='', column_title='Total Deaths per 100k')
    continents_rate_row = trend_confirmed_cases | trend_confirmed_cases_recent | trend_confirmed_deaths | trend_confirmed_deaths_recent | total_confirmed_cases | total_confirmed_deaths
    
    chart = trend_row & continents_row & continents_rate_row
    
    chart = chart.resolve_scale(color='independent')
    chart = configure_trend_chart(chart)

    return chart

footer = 'Area charts and bar charts from daily data; line charts from 7 day rolling mean'

chart = get_world_overview_chart(df_covid_time_series_world)
chart.display()
print(footer)

if export_slides:
    chart.save(saved_image_name)

    # Add a new slide ready for chart and set title text
    slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
    prs.set_text(slide, placeholder_name='title', text=slide_title)
    prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
    prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
slide_title = 'COVID-19 Trend - Absolute'
display_h1(slide_title)

top_n = 10

def trend_dashboard_charts(df, top_n):
    df = df.copy()
    
    chart_width_large = 300
    chart_width_small = 150
    chart_width_smaller = 100
    chart_height = 110
    
    sort_field = 'deaths'
    
    df_totals = get_totals_from_time_series(df)
    df_continents = df_continents_time_series_summary(df)

    chart = alt.vconcat()
    first_row = True
    x_min = df['date'].min()
    x_max = df['date'].max()
    
    for continent in df['continent'].unique():
        df_filtered = df[df['continent']==continent]
        if not top_n is None:
            df_totals_filtered = df_totals.copy()
            df_totals_filtered = df_totals_filtered[df_totals_filtered['continent']==continent]
            df_totals_filtered = df_totals_filtered.sort_values(by=sort_field, ascending=False)
            df_totals_filtered = df_totals_filtered[:top_n]
            combined_key_top_n = df_totals_filtered['Combined_Key']
            df_filtered = df_filtered[df_filtered['Combined_Key'].isin(combined_key_top_n)] # would like to put it in the order of highest totals

        trend_confirmed_cases  = trend_chart(df_filtered, metric='confirmed', category='Combined_Key', x_min=x_min, x_max=x_max, 
                                             chart_width=chart_width_large, chart_height=chart_height, 
                                             area_chart=False, row_title=continent, column_title='Daily Cases' if first_row else '')
        trend_confirmed_cases_recent  = trend_chart(df_filtered, metric='confirmed', category='Combined_Key', x_min=x_min, x_max=x_max, filter_last_weeks=4,
                                             chart_width=chart_width_smaller, chart_height=chart_height, 
                                             area_chart=False, row_title='', column_title='Cases 4wks' if first_row else '')

        trend_confirmed_deaths = trend_chart(df_filtered, metric='deaths',    category='Combined_Key', x_min=x_min, x_max=x_max, 
                                             chart_width=chart_width_large, chart_height=chart_height, 
                                             area_chart=False, row_title='', column_title='Daily Deaths' if first_row else '')
        trend_confirmed_deaths_recent = trend_chart(df_filtered, metric='deaths',    category='Combined_Key', x_min=x_min, x_max=x_max,  filter_last_weeks=4,
                                             chart_width=chart_width_smaller, chart_height=chart_height, 
                                             area_chart=False, row_title='', column_title='Deaths 4wks' if first_row else '')

        total_confirmed_cases  = bar_chart(df_filtered, metric='confirmed', category='Combined_Key', rollup='total', numbers_overlaid=False,
                                             chart_width=chart_width_small, chart_height=chart_height, 
                                           row_title='', column_title='Total Cases' if first_row else '')
        total_confirmed_deaths  = bar_chart(df_filtered, metric='deaths',   category='Combined_Key', rollup='total', numbers_overlaid=False,
                                             chart_width=chart_width_small, chart_height=chart_height, 
                                            row_title='', column_title='Total Deaths' if first_row else '')

        first_row = False

        row = trend_confirmed_cases | trend_confirmed_cases_recent | trend_confirmed_deaths | trend_confirmed_deaths_recent | total_confirmed_cases | total_confirmed_deaths
        chart = chart & row

    chart = chart.resolve_scale(color='independent')
    chart = configure_trend_chart(chart)

    
    return chart

footer = f'Per continent, top {top_n} by total deaths; trend charts using 7 day rolling mean'

chart = trend_dashboard_charts(df_covid_time_series_world, top_n)
chart.display()
print(footer)

if export_slides:
    chart.save(saved_image_name)

    # Add a new slide ready for chart and set title text
    slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
    prs.set_text(slide, placeholder_name='title', text=slide_title)
    prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
    prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
slide_title = 'COVID-19 Vaccinations - World View'
display_h1(slide_title)

country_population_threshold_to_include = 1000000

def vaccine_trend_dashboard_charts_horizontal(df, country_population_threshold_to_include):
    df = df.copy()
    
    chart_width_large = 300
    chart_width_small = 150
    chart_width_smaller = 100
    chart_height_small = 110
    chart_height_medium = 170
    chart_height_large = 300
    
#     sort_field = 'deaths'
    
    chart = alt.hconcat()
    first_col = True
    x_min = df['date'].min()
    x_max = df['date'].max()
    
    if country_population_threshold_to_include:
        df = df[df['population'] > country_population_threshold_to_include]

    y_max_vaccinated = df['people_vaccinated_per_hundred'].max()
    y_max_fully_vaccinated = df['people_fully_vaccinated_per_hundred'].max()
    
    for continent in df['continent'].unique():
        df_filtered = df[df['continent']==continent]
        
        trend_people_vaccinated  = trend_chart(df_filtered, metric='people_vaccinated_per_hundred', category='country', x_min=x_min, x_max=x_max, y_max=y_max_vaccinated,
                                             chart_width=chart_width_small, chart_height=chart_height_small, 
                                             plot_7_day_average=False, forward_fill=True,
                                             area_chart=False, row_title='% Vaccinated' if first_col else '', column_title=continent)

        trend_people_fully_vaccinated  = trend_chart(df_filtered, metric='people_fully_vaccinated_per_hundred', category='country', x_min=x_min, x_max=x_max, y_max=y_max_fully_vaccinated,
                                             chart_width=chart_width_small, chart_height=chart_height_small, 
                                             plot_7_day_average=False, forward_fill=True,
                                             area_chart=False, row_title='% Fully Vacc.' if first_col else '', column_title='')

        total_people_vaccinated  = bar_chart(df_filtered, metric='people_vaccinated_per_hundred', sort_field='people_vaccinated_per_hundred', category='country', 
                                             rollup='latest', numbers_overlaid=False, forward_fill=True,
                                             chart_width=chart_width_small, chart_height=chart_height_large, 
                                             row_title='% Vaccinated' if first_col else '', column_title='')

        total_people_fully_vaccinated  = bar_chart(df_filtered, metric='people_fully_vaccinated_per_hundred', sort_field='people_fully_vaccinated_per_hundred', category='country', 
                                             rollup='latest', numbers_overlaid=False, forward_fill=True,
                                             chart_width=chart_width_small, chart_height=chart_height_large, 
                                             row_title='% Fully Vaccinated' if first_col else '', column_title='')

        first_col = False

        col = trend_people_vaccinated & trend_people_fully_vaccinated & total_people_vaccinated & total_people_fully_vaccinated
        chart = chart | col

    chart = chart.resolve_scale(color='independent')
    chart = configure_trend_chart(chart)

    
    return chart

footer = f'Per continent, vaccinations per country with population at least {country_population_threshold_to_include/1000000} million. Vaccinated is with at least 1 dose.'

chart = vaccine_trend_dashboard_charts_horizontal(df_world_vaccines, country_population_threshold_to_include)
chart.display()
print(footer)

if export_slides:
    chart.save(saved_image_name)

    # Add a new slide ready for chart and set title text
    slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
    prs.set_text(slide, placeholder_name='title', text=slide_title)
    prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
    prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
slide_title = 'Covid-19 UK'
display_h1(slide_title)

def configure_trend_chart_uk(chart):
    chart = chart.configure_axisY(
        titleColor='blue',
        titleFontSize=20,
    )
    
    chart = chart.configure_title(
        fontSize=20,
        anchor='middle',
        color='blue'
    )
    
    return chart

def get_uk_overview_chart(df):
    df = df.copy()
    
    chart_width_large = 300
    chart_width_small = 150
    chart_width_smaller = 110
    chart_height = 150

    df['date]'] = pd.to_datetime(df['date'])

    x_min = df['date'].min()
    x_max = df['date'].max()
    
    trend_confirmed_cases  = trend_chart(df, metric='confirmed', category='areaType', x_min=x_min, x_max=x_max, 
                                         chart_width=chart_width_large, chart_height=chart_height, 
                                         row_title='UK', column_title='Daily Cases')
    trend_confirmed_cases_recent  = trend_chart(df, metric='confirmed', category='areaType', x_min=x_min, x_max=x_max, filter_last_weeks=4,
                                                chart_width=chart_width_smaller, chart_height=chart_height, 
                                         row_title='', column_title='Cases 4wks')
    trend_confirmed_deaths = trend_chart(df, metric='deaths',    category='areaType', x_min=x_min, x_max=x_max,
                                         chart_width=chart_width_large, chart_height=chart_height, 
                                         row_title='', column_title='Daily Deaths')
    trend_confirmed_deaths_recent = trend_chart(df, metric='deaths',    category='areaType', x_min=x_min, x_max=x_max, filter_last_weeks=4,
                                                chart_width=chart_width_smaller, chart_height=chart_height, 
                                         row_title='', column_title='Daily Deaths')
    total_confirmed_cases  = bar_chart(df, metric='confirmed', category='areaType', rollup='total', numbers_overlaid=True,
                                       chart_width=chart_width_small, chart_height=chart_height, 
                                       row_title='', column_title='Total Cases')
    total_confirmed_deaths  = bar_chart(df, metric='deaths',   category='areaType', rollup='total', numbers_overlaid=True,
                                        chart_width=chart_width_small, chart_height=chart_height, 
                                        row_title='', column_title='Total Deaths')
    uk_trend_row = trend_confirmed_cases | trend_confirmed_cases_recent | trend_confirmed_deaths | trend_confirmed_deaths_recent | total_confirmed_cases | total_confirmed_deaths
    
    trend_confirmed_cases   = trend_chart(df, metric='confirmed', category='areaName', x_min=x_min, x_max=x_max, 
                                          chart_width=chart_width_large, chart_height=chart_height, 
                                          area_chart=False, row_title='Nations', column_title='Daily Cases')
    trend_confirmed_cases_recent   = trend_chart(df, metric='confirmed', category='areaName', x_min=x_min, x_max=x_max, filter_last_weeks=4,
                                          chart_width=chart_width_smaller, chart_height=chart_height, 
                                          area_chart=False, row_title='', column_title='Cases 4wks')
    trend_confirmed_deaths  = trend_chart(df, metric='deaths',    category='areaName', x_min=x_min, x_max=x_max, 
                                          chart_width=chart_width_large, chart_height=chart_height, 
                                          remove_trailing_zeros=True,
                                          area_chart=False, row_title='', column_title='Daily Deaths')
    trend_confirmed_deaths_recent  = trend_chart(df, metric='deaths',    category='areaName', x_min=x_min, x_max=x_max, filter_last_weeks=4,
                                          chart_width=chart_width_smaller, chart_height=chart_height, 
                                          remove_trailing_zeros=True,
                                          area_chart=False, row_title='', column_title='Deaths 4wks')
    total_confirmed_cases  = bar_chart(df, metric='confirmed', category='areaName', rollup='total', numbers_overlaid=False,
                                       chart_width=chart_width_small, chart_height=chart_height, 
                                       row_title='', column_title='Total Cases')
    total_confirmed_deaths  = bar_chart(df, metric='deaths',   category='areaName', rollup='total', numbers_overlaid=False,
                                        chart_width=chart_width_small, chart_height=chart_height, 
                                        row_title='', column_title='Total Deaths')
    uk_nations_row = trend_confirmed_cases | trend_confirmed_cases_recent | trend_confirmed_deaths | trend_confirmed_deaths_recent | total_confirmed_cases | total_confirmed_deaths
    
    trend_confirmed_cases   = trend_chart(df, metric='confirmedRate', category='areaName', x_min=x_min, x_max=x_max, 
                                          chart_width=chart_width_large, chart_height=chart_height, 
                                          area_chart=False, remove_trailing_zeros=True, plot_7_day_average=False,
                                          row_title='Nations', column_title='Daily Cases per 100k')
    trend_confirmed_cases_recent   = trend_chart(df, metric='confirmedRate', category='areaName', x_min=x_min, x_max=x_max, filter_last_weeks=4,
                                          chart_width=chart_width_smaller, chart_height=chart_height, 
                                          area_chart=False, remove_trailing_zeros=True, plot_7_day_average=False,
                                          row_title='', column_title='Cases 4wks/100k')
    trend_confirmed_deaths  = trend_chart(df, metric='deathsRate',    category='areaName', x_min=x_min, x_max=x_max, 
                                          chart_width=chart_width_large, chart_height=chart_height, 
                                          area_chart=False, remove_trailing_zeros=True, plot_7_day_average=False,
                                          row_title='', column_title='Daily Deaths per 100k')
    trend_confirmed_deaths_recent  = trend_chart(df, metric='deathsRate',    category='areaName', x_min=x_min, x_max=x_max, filter_last_weeks=4,
                                          chart_width=chart_width_smaller, chart_height=chart_height, 
                                          area_chart=False, remove_trailing_zeros=True, plot_7_day_average=False,
                                          row_title='', column_title='Deaths 4wks/100k')
    total_confirmed_cases  = bar_chart(df, metric='confirmedRate', sort_field='deathsRate', category='areaName', rollup='total', numbers_overlaid=False,
                                       chart_width=chart_width_small, chart_height=chart_height, 
                                       row_title='', column_title='Total Cases per 100k')
    total_confirmed_deaths  = bar_chart(df, metric='deathsRate',   sort_field='deathsRate', category='areaName', rollup='total', numbers_overlaid=False,
                                        chart_width=chart_width_small, chart_height=chart_height, 
                                        row_title='', column_title='Total Deaths per 100k')
    uk_nations_rate_row = trend_confirmed_cases | trend_confirmed_cases_recent | trend_confirmed_deaths | trend_confirmed_deaths_recent | total_confirmed_cases | total_confirmed_deaths
    
    chart = uk_trend_row & uk_nations_row & uk_nations_rate_row
    
    chart = chart.resolve_scale(color='independent')
    chart = configure_trend_chart_uk(chart)

    return chart

footer = 'Area charts and bar charts from daily data; line charts from 7 day rolling mean'

print('England: Data include only pillar 1 cases until 2 July, from when pillar 2 cases are also included.')

chart = get_uk_overview_chart(df_uk_overview)
chart.display()
print(footer)

if export_slides:
    chart.save(saved_image_name)

    # Add a new slide ready for chart and set title text
    slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
    prs.set_text(slide, placeholder_name='title', text=slide_title)
    prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
    prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
slide_title = 'England Authorities with Highest Local Case Rates'
display_h1(slide_title)
footer = 'Case rates per 100k population, from highest weekly Middle Layer Super Output Areas (MSOA) data'

chart_height = 200
chart_width  = 300

def df_latest_max_with_delta(df, series_field='date', group_for_max='areaName', group_for_delta='areaName', value_field='value'):
    df = df.copy()

    df = df.groupby(group_for_max).agg({value_field: 'max'}).reset_index()

    df['delta'] = df.groupby(group_for_delta)[value_field].transform(lambda x: x.diff())
    latest_date = df[series_field].max()
    df = df[df[series_field]==latest_date]

    return df

def get_rate_bar_chart(df, category='index', metric='value', colour_field='delta', x_min=None, x_max=None, colour_field_max=None, row_title='', column_title='', top_n=20):
    df = df.copy()

    df = df.reset_index() # to ensure important fields are accessible in columns rather than hidden in index

    sort_order=None
    if not top_n is None:
        sort_order = list(df.sort_values(by=metric, ascending=False)[category].unique())
        sort_order = sort_order[:top_n]
        df = df[df[category].isin(sort_order)]

    x_min = 0
    if not x_max:
        x_max = df[metric].max()
        
    if not colour_field_max:
        colour_field_max = df[colour_field].max()

    legend_list = ['-800 or less', '-400 to -800', '0 to -400', '0 to +400', '+400 to +800', '+800 or more']
    # CSS colours: https://www.w3schools.com/colors/colors_shades.asp
    legend_colour_list = ['green', 'darkseagreen', 'palegreen', 'moccasin', 'orange', 'red']

    df['legend'] = pd.cut(df[colour_field], 
                          [-np.inf, -800, -400, 0, 400, 800, +np.inf], 
                          labels=legend_list)
    
    bars = alt.Chart().mark_bar(
    ).encode(
        x=alt.X(f'{metric}:Q', scale=alt.Scale(domain=[x_min, x_max])),
        y=alt.Y(f'{category}:N', sort=sort_order, title=row_title), # sort='-x' to sort locally in this chart 
        color=alt.Color('legend:N', 
                        scale=alt.Scale(domain=legend_list, range=legend_colour_list),
                        legend=alt.Legend(title="Change since previous week"),
                       ),

        tooltip=[alt.Tooltip(f'{category}:N'),
                 alt.Tooltip(f'{metric}:Q', format=',d'),
                 alt.Tooltip(f'{colour_field}:Q', format=',d')]

    )

    text = bars.mark_text(
        align='left',
        baseline='middle',
        dx=3  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
        text=alt.Text(f'{metric}:Q', format=',d') #,.2r gives rounded to 2 significant figures
    )
    
    chart = alt.layer(bars, text, data=df)
    chart = chart.properties(title=column_title, height=chart_height, width=chart_width)
    
    return chart

def facet_wrap(subplts, plots_per_row):
    rows = [subplts[i:i+plots_per_row] for i in range(0, len(subplts), plots_per_row)]
    compound_chart = alt.vconcat()
    for r in rows:
        rowplot = alt.vconcat() #start a new row
        for item in r:
            rowplot |= item #add suplot to current row as a new column
        compound_chart &= rowplot # add the entire row of plots as a new row

    return compound_chart

def configure_chart(chart):
    chart = chart.configure_axisY(
        titleColor='blue',
        titleFontSize=20,
    )
    
    chart = chart.configure_title(
        fontSize=20,
        anchor='middle',
        color='blue'
    )
    return chart

def get_rate_charts_per_region(df, top_n=20, plots_per_row=3):
    df = df_latest_max_with_delta(df, group_for_max=['date', 'RegionName', 'LTLAName'], group_for_delta=['RegionName', 'LTLAName'])
    
    sort_field = 'value'
    category_field = 'LTLAName'
    
    selection_list_top_n = []
    if top_n is None:
        df_filtered = df.copy()
    else:
        for region in df['RegionName'].unique():
            df_filtered = df[df['RegionName']==region]
            if (not df_filtered.empty) and (sort_field in df_filtered):
                df_filtered = df_filtered.sort_values(by=sort_field, ascending=False)
                df_filtered = df_filtered[:top_n]
                selection_top_n = list(df_filtered[category_field].unique())
                selection_list_top_n = selection_list_top_n + selection_top_n
        df_filtered = df[df[category_field].isin(selection_list_top_n)] # would like to put it in the order of highest totals

    value_max = df_filtered['value'].max()
    colour_field_max = 1000#df_filtered['delta'].max()

    rates_chart_list = []
    for region in df['RegionName'].unique():
        df_facet = df_filtered[df_filtered['RegionName']==region]
        sorted_order = df_facet.sort_values(by=sort_field, ascending=False)[category_field].unique()

        rates_chart = get_rate_bar_chart(df_facet, category=category_field, metric='value', colour_field='delta', x_max=value_max, colour_field_max=colour_field_max, column_title=region, top_n=top_n)
        rates_chart_list.append(rates_chart)

    chart = facet_wrap(rates_chart_list, plots_per_row=plots_per_row)#.properties(title='MSOA Rates per Region')
    chart = configure_chart(chart)

    return chart
    
chart = get_rate_charts_per_region(df_uk_msoa_with_regions, top_n=10)
print('Latest date:', df_uk_msoa_with_regions['date'].max())
chart.display()

print(footer)

if export_slides:
    chart.save(saved_image_name)

    # Add a new slide ready for chart and set title text
    slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
    prs.set_text(slide, placeholder_name='title', text=slide_title)
    prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
    prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
slide_title = 'England Cases by Age Group'
display_h1(slide_title)

footer = 'Cases by age group, as 7 day rolling average'

def df_get_case_age_breakdown_trend(df, field): # maleCasesValue, maleCasesRate, femaleCasesValue, femaleCasesRate
    df = df.copy()
    df = df.reset_index()
    
    df = df[df[field].notnull()]
    df_age_data = pd.DataFrame(df[field].to_list())
#     df_age_data = df_age_data.rename(columns={'5_to_9':'05_to_09', '0_to_4':'00_to_04'})
    df_age_data = df_age_data.drop(['00_59', '60+'], axis='columns', errors='ignore')
    cols = sorted(df_age_data.columns.tolist())
    df_age_data = df_age_data[cols]
    
    index_cols = ['date']
    df = pd.concat([df[index_cols], df_age_data], axis='columns')
    
    df = df.melt('date', var_name='ageRange', value_name='value')
    df = df.set_index(index_cols)

    return df

def get_age_trend_chart(df):
    chart_width = 300
    chart_width_small = 150
    chart_width_smaller = 110
    chart_height = 300

    df_age_cases = df_get_case_age_breakdown_trend(df, 'casesByAge')
    df_age_deaths = df_get_case_age_breakdown_trend(df, 'deathsByAge')
    chart_age_cases = trend_chart(df_age_cases, metric='value', category='ageRange', 
                                  area_chart=False, remove_trailing_zeros=True, 
                                  chart_width=chart_width, chart_height=chart_height, 
                                  row_title='Absolute', column_title='Daily Cases')
    chart_age_cases_recent = trend_chart(df_age_cases, metric='value', category='ageRange', filter_last_weeks=4,
                                  area_chart=False, remove_trailing_zeros=True, 
                                  chart_width=chart_width_smaller, chart_height=chart_height, 
                                  row_title='', column_title='Cases 4wks')
    chart_age_deaths = trend_chart(df_age_deaths, metric='value', category='ageRange', 
                                  area_chart=False, remove_trailing_zeros=True, 
                                  chart_width=chart_width, chart_height=chart_height, 
                                  row_title='', column_title='Daily Deaths')
    chart_age_deaths_recent = trend_chart(df_age_deaths, metric='value', category='ageRange', filter_last_weeks=4,
                                  area_chart=False, remove_trailing_zeros=True, 
                                  chart_width=chart_width_smaller, chart_height=chart_height, 
                                  row_title='', column_title='Deaths 4wks')
    total_age_cases = bar_chart(df_age_cases, metric='value', category='ageRange', rollup='total', numbers_overlaid=False,
                                  chart_width=chart_width_small, chart_height=chart_height,
                                  row_title='', column_title='Total Cases')
    total_age_deaths = bar_chart(df_age_deaths, metric='value', category='ageRange', rollup='total', numbers_overlaid=False,
                                  chart_width=chart_width_small, chart_height=chart_height, 
                                  row_title='', column_title='Total Deaths')
    row1 = chart_age_cases | chart_age_cases_recent | chart_age_deaths | chart_age_deaths_recent | total_age_cases | total_age_deaths

    
    df_age_cases = df_get_case_age_breakdown_trend(df, 'casesRateByAge')
    df_age_deaths = df_get_case_age_breakdown_trend(df, 'deathsRateByAge')
    chart_age_cases = trend_chart(df_age_cases, metric='value', category='ageRange', 
                                  area_chart=False, remove_trailing_zeros=True, plot_7_day_average=False,
                                  chart_width=chart_width, chart_height=chart_height, 
                                  row_title='Per 100k Population', column_title='Daily Cases per 100k')
    chart_age_cases_recent = trend_chart(df_age_cases, metric='value', category='ageRange', filter_last_weeks=4,
                                  area_chart=False, remove_trailing_zeros=True, plot_7_day_average=False,
                                  chart_width=chart_width_smaller, chart_height=chart_height, 
                                  row_title='', column_title='Cases 4wks/100k')
    chart_age_deaths = trend_chart(df_age_deaths, metric='value', category='ageRange', 
                                  area_chart=False, remove_trailing_zeros=True, plot_7_day_average=False,
                                  chart_width=chart_width, chart_height=chart_height, 
                                  row_title='', column_title='Daily Deaths per 100k')
    chart_age_deaths_recent = trend_chart(df_age_deaths, metric='value', category='ageRange', filter_last_weeks=4,
                                  area_chart=False, remove_trailing_zeros=True, plot_7_day_average=False,
                                  chart_width=chart_width_smaller, chart_height=chart_height, 
                                  row_title='', column_title='Deaths 4wks/100k')
    total_age_cases = bar_chart(df_age_cases, metric='value', category='ageRange', rollup='total', numbers_overlaid=False,
                                  chart_width=chart_width_small, chart_height=chart_height, 
                                  row_title='', column_title='Total Cases per 100k')
    total_age_deaths = bar_chart(df_age_deaths, metric='value', category='ageRange', rollup='total', numbers_overlaid=False,
                                  chart_width=chart_width_small, chart_height=chart_height, 
                                  row_title='', column_title='Total Deaths per 100k')
    row2 = chart_age_cases | chart_age_cases_recent | chart_age_deaths | chart_age_deaths_recent | total_age_cases | total_age_deaths
    
    chart = row1 & row2
    
    chart = configure_trend_chart(chart)
    
    return chart

if not df_uk_age.empty:
    chart = get_age_trend_chart(df_uk_age)
    chart.display()

    print(footer)

    if export_slides:
        chart.save(saved_image_name)

        # Add a new slide ready for chart and set title text
        slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
        prs.set_text(slide, placeholder_name='title', text=slide_title)
        prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
        prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
slide_title = 'England Hospitals'
display_h1(slide_title)
footer = 'Area charts and bar charts from daily data; line charts from 7 day rolling mean'

print('England: Data include only pillar 1 cases until 2 July, from when pillar 2 cases are also included.')

def df_get_case_age_breakdown_trend_hospital(df, field): # maleCasesValue, maleCasesRate, femaleCasesValue, femaleCasesRate
    df = df.copy()
    df = df.reset_index()
    
    df = df.filter(['date', field])[df[field].notnull()]
    df = pd.concat([df.drop([field], axis=1), df[field].apply(pd.Series)], axis=1)
    df = df.rename(columns={'6_to_17':'06_to_17', '0_to_5':'00_to_05'})

    cols = sorted(df.columns.tolist())
    df = df[cols]
    
    index_cols = ['date']
    df = df.melt('date', var_name='ageRange', value_name='value')

    return df

def get_uk_hospital_chart(df, df_age):
    df = df.copy()
    
    chart_width_large = 500
    chart_width_small = 300
    chart_width_smaller = 110
    chart_height      = 150
    
    x_min = df['date'].min()
    x_max = df['date'].max()
    
    df_hospital_cases = df_create_categories(df, index_col='date', columns=['date', 'hospitalCases', 'covidOccupiedMVBeds'], category_field_name='category', value_field_name='value')

    df['cumAdmissionsByAgeValue'] = df_age["cumAdmissionsByAge"].apply(lambda x : unpack_list_of_dict(x, 'age', 'value') )
    df_hospital_cases_age = df_get_case_age_breakdown_trend_hospital(df, 'cumAdmissionsByAgeValue')
    df_hospital_cases_age = df_delta_from_cumulative(df_hospital_cases_age, metric='value', category='ageRange', new_metric='delta_value')

    trend_hospital_admissions  = trend_chart(df, metric='hospitalAdmissions', category='areaType', x_min=x_min, x_max=x_max, 
                                             chart_width=chart_width_large, chart_height=chart_height, 
                                             row_title='Total', column_title='Daily Admissions')
    trend_hospital_admissions_recent  = trend_chart(df, metric='hospitalAdmissions', category='areaType', x_min=x_min, x_max=x_max, filter_last_weeks=4,
                                             chart_width=chart_width_smaller, chart_height=chart_height, 
                                             row_title='', column_title='Admissions 4wks')
    total_hospital_admissions  = bar_chart(df, metric='hospitalAdmissions', category='areaType', rollup='total', numbers_overlaid=True,
                                           chart_width=chart_width_small, chart_height=chart_height, 
                                           row_title='', column_title='Total Admissions')
    hospital_admissions_row = trend_hospital_admissions | trend_hospital_admissions_recent | total_hospital_admissions
    
    trend_hospital_admissions_age = trend_chart(df_hospital_cases_age, metric='delta_value', category='ageRange', x_min=x_min, x_max=x_max, 
                                                area_chart=False, remove_trailing_zeros=True, plot_7_day_average=True, 
                                                chart_width=chart_width_large, chart_height=chart_height, 
                                                row_title='By Age Group', column_title='Daily Admissions')
    trend_hospital_admissions_age_recent = trend_chart(df_hospital_cases_age, metric='delta_value', category='ageRange', x_min=x_min, x_max=x_max, filter_last_weeks=4,
                                                area_chart=False, remove_trailing_zeros=True, plot_7_day_average=True, 
                                                chart_width=chart_width_smaller, chart_height=chart_height, 
                                                row_title='', column_title='Admissions 4wks')
    total_hospital_admissions_age = bar_chart(df_hospital_cases_age, metric='delta_value', category='ageRange', rollup='total', numbers_overlaid=False,
                                              chart_width=chart_width_small, chart_height=chart_height, 
                                              row_title='', column_title='')
    hospital_admissions_age_row = trend_hospital_admissions_age | trend_hospital_admissions_age_recent | total_hospital_admissions_age
    
    trend_hospital_cases   = trend_chart(df_hospital_cases, metric='value', category='category', x_min=x_min, x_max=x_max, 
                                         area_chart=False, remove_trailing_zeros=True, plot_7_day_average=False,
                                         chart_width=chart_width_large, chart_height=chart_height, 
                                         row_title='Total', column_title='Hospital Cases')
    trend_hospital_cases_recent   = trend_chart(df_hospital_cases, metric='value', category='category', x_min=x_min, x_max=x_max, filter_last_weeks=4,
                                         area_chart=False, remove_trailing_zeros=True, plot_7_day_average=False,
                                         chart_width=chart_width_smaller, chart_height=chart_height, 
                                         row_title='', column_title='Cases 4wks')
    latest_hospital_cases  = bar_chart(df_hospital_cases, metric='value', sort_field='value', category='category', rollup='latest', numbers_overlaid=False,
                                       chart_width=chart_width_small, chart_height=chart_height, 
                                       row_title='', column_title='Latest')
    hospital_cases_row = trend_hospital_cases | trend_hospital_cases_recent | latest_hospital_cases
    
    chart = hospital_admissions_row & hospital_admissions_age_row & hospital_cases_row
    
    chart = chart.resolve_scale(color='independent')
    chart = configure_trend_chart_uk(chart)

    return chart

if not df_uk_hospital_england.empty:
    chart = get_uk_hospital_chart(df_uk_hospital_england, df_uk_hospital_age_england)
    chart.display()
    print(footer)

    if export_slides:
        chart.save(saved_image_name)

        # Add a new slide ready for chart and set title text
        slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
        prs.set_text(slide, placeholder_name='title', text=slide_title)
        prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
        prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
slide_title = 'UK Covid Vaccinations'
display_h1(slide_title)
footer = 'Area charts and bar charts from daily data; line charts from 7 day rolling mean'

# def df_cumulative_from_delta(df, metric, category, new_metric):
def get_uk_vaccinations_chart(df):
    df = df.copy()
    
    chart_width_large = 300
    chart_width_small = 200
    chart_height      = 100
    
    x_min = df['date'].min()
    x_max = df['date'].max()
    
    trend_total_first  = trend_chart(df, metric='cumVaccineFirstDose', category='areaType', x_min=x_min, x_max=x_max, y_min=0,
                               chart_width=chart_width_large, chart_height=chart_height, 
                               area_chart=False, plot_7_day_average=False,
                               row_title='UK', column_title='Cumulative First Dose')
    trend_total_second  = trend_chart(df, metric='cumVaccineSecondDose', category='areaType', x_min=x_min, x_max=x_max, y_min=0,
                               chart_width=chart_width_large, chart_height=chart_height, 
                               area_chart=False, plot_7_day_average=False,
                               row_title='', column_title='Cumulative Second Dose')
    total_first  = bar_chart(df, metric='cumVaccineFirstDose', category='areaType', rollup='latest', numbers_overlaid=True,
                                           chart_width=chart_width_small, chart_height=chart_height, 
                                           row_title='', column_title='Total First Dose')
    total_second  = bar_chart(df, metric='cumVaccineSecondDose', category='areaType', rollup='latest', numbers_overlaid=True,
                                           chart_width=chart_width_small, chart_height=chart_height, 
                                           row_title='', column_title='Total Second Dose')

    total_row = trend_total_first | trend_total_second | total_first | total_second
    
    trend_nation_total_first  = trend_chart(df, metric='cumVaccineFirstDose', category='areaName', x_min=x_min, x_max=x_max, y_min=0,
                               chart_width=chart_width_large, chart_height=chart_height, 
                               area_chart=False, plot_7_day_average=False,
                               row_title='Nations', column_title='')
    trend_nation_total_second  = trend_chart(df, metric='cumVaccineSecondDose', category='areaName', x_min=x_min, x_max=x_max, y_min=0,
                               chart_width=chart_width_large, chart_height=chart_height, 
                               area_chart=False, plot_7_day_average=False,
                               row_title='', column_title='')
    total_nation_first  = bar_chart(df, metric='cumVaccineFirstDose', category='areaName', 
                              sort_field='cumVaccineFirstDose', rollup='latest', numbers_overlaid=False,
                               chart_width=chart_width_small, chart_height=chart_height, 
                               row_title='', column_title='')
    total_nation_second  = bar_chart(df, metric='cumVaccineSecondDose', category='areaName', 
                              sort_field='cumVaccineSecondDose', rollup='latest', numbers_overlaid=False,
                               chart_width=chart_width_small, chart_height=chart_height, 
                               row_title='', column_title='')
    total_nation_row = trend_nation_total_first | trend_nation_total_second | total_nation_first | total_nation_second


    trend_total_percent_first  = trend_chart(df, metric='cumVaccineFirstDose', category='areaType', x_min=x_min, x_max=x_max, y_min=0,
                               chart_width=chart_width_large, chart_height=chart_height, 
                               area_chart=False, plot_7_day_average=False, per_population_percent=True,
                               row_title='UK', column_title='Cumulative First Dose by % Population')
    trend_total_percent_second  = trend_chart(df, metric='cumVaccineSecondDose', category='areaType', x_min=x_min, x_max=x_max, y_min=0,
                               chart_width=chart_width_large, chart_height=chart_height, 
                               area_chart=False, plot_7_day_average=False, per_population_percent=True,
                               row_title='', column_title='Cumulative Second Dose by % Population')
    total_percent_first  = bar_chart(df, metric='cumVaccineFirstDose', category='areaType', rollup='latest', numbers_overlaid=True, per_population_percent=True,
                                           chart_width=chart_width_small, chart_height=chart_height, 
                                           row_title='', column_title='Total First Dose by % Population')
    total_percent_second  = bar_chart(df, metric='cumVaccineSecondDose', category='areaType', rollup='latest', numbers_overlaid=True, per_population_percent=True,
                                           chart_width=chart_width_small, chart_height=chart_height, 
                                           row_title='', column_title='Total Second Dose by % Population')
    total_row_percent = trend_total_percent_first | trend_total_percent_second | total_percent_first | total_percent_second

    trend_nation_total_percent_first  = trend_chart(df, metric='cumVaccineFirstDose', category='areaName', x_min=x_min, x_max=x_max, y_min=0,
                               chart_width=chart_width_large, chart_height=chart_height, 
                               area_chart=False, plot_7_day_average=False, per_population_percent=True,
                               row_title='Nations', column_title='')
    trend_nation_total_percent_second  = trend_chart(df, metric='cumVaccineSecondDose', category='areaName', x_min=x_min, x_max=x_max, y_min=0,
                               chart_width=chart_width_large, chart_height=chart_height, 
                               area_chart=False, plot_7_day_average=False, per_population_percent=True,
                               row_title='', column_title='')
    total_nation_percent_first  = bar_chart(df, metric='cumVaccineFirstDose', category='areaName', 
                              sort_field='cumVaccineFirstDose', rollup='latest', numbers_overlaid=False, per_population_percent=True,
                               chart_width=chart_width_small, chart_height=chart_height, 
                               row_title='', column_title='')
    total_nation_percent_second  = bar_chart(df, metric='cumVaccineSecondDose', category='areaName', 
                              sort_field='cumVaccineSecondDose', rollup='latest', numbers_overlaid=False, per_population_percent=True,
                               chart_width=chart_width_small, chart_height=chart_height, 
                               row_title='', column_title='')
    total_nation_row_percent = trend_nation_total_percent_first | trend_nation_total_percent_second | total_nation_percent_first | total_nation_percent_second

    chart = total_row & total_nation_row & total_row_percent & total_nation_row_percent
    
    chart = chart.resolve_scale(color='independent')
    chart = configure_trend_chart_uk(chart)

    return chart

if len(df_uk_vaccinations)>0:
    chart = get_uk_vaccinations_chart(df_uk_vaccinations)
    chart.display()
else:
    print("No vaccinations data found")
print(footer)

if export_slides:
    chart.save(saved_image_name)

    # Add a new slide ready for chart and set title text
    slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
    prs.set_text(slide, placeholder_name='title', text=slide_title)
    prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
    prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
slide_title = 'England Lower Tier Authorities'
display_h1(slide_title)
top_n = 15

chart_height = 250
trend_chart_width = 350
totals_chart_width = 200

def facet_wrap(subplts, plots_per_row):
    rows = [subplts[i:i+plots_per_row] for i in range(0, len(subplts), plots_per_row)]
    compound_chart = alt.vconcat()
    for r in rows:
        rowplot = alt.vconcat() #start a new row
        for item in r:
            rowplot |= item #add suplot to current row as a new column
        compound_chart &= rowplot # add the entire row of plots as a new row

    return compound_chart

def df_get_daily_average_and_totals(df, index_col_list, metric):
    df_daily = df.copy()
    df_daily = df_daily.dropna(axis='index', subset=[metric])
    df_daily = df_daily.set_index(index_col_list, append=True)
    df_daily[metric] = df_daily.groupby(index_col_list)[metric].transform(lambda x: x.diff()).clip(lower=0)
    df_totals = pd.DataFrame()
    if not df_daily.empty:
        df_totals = df_daily.groupby(index_col_list).agg({metric:'sum'})
        df_daily['weekAvg'] = df_daily.groupby(index_col_list)[metric].rolling(window=7, min_periods=1).mean().reset_index(0, drop=True)
    return df_daily.reset_index(), df_totals.reset_index()
    
def uk_regions_trend_dashboard_charts(df, filter_area_name_list=None, normalised=False, multiple_facets=True, plots_per_row=3, scale_type='linear', x_metric='date', top_n=None, chart_height=250, trend_chart_width=350, totals_chart_width=200):
    df = df.copy()
    
    df = df[df['regionName'].notnull()]
    
    if not normalised:
        confirmed_field = 'confirmed'
        deaths_field = 'deaths'
        normalised_title_string = ''
        plot_7_day_average=True,
    else:
        confirmed_field = 'confirmedRate'
        deaths_field = 'deathsRate' # not in data yet
        normalised_title_string = ' per 100k Population'
        plot_7_day_average=False,
    sort_field = confirmed_field
    
    chart = alt.vconcat()
    first_row = True
    if filter_area_name_list:
        df = df[df['areaName'].isin(filter_area_name_list)]
    
    x_min = df[x_metric].min()
    x_max = df[x_metric].max()
    
    if not multiple_facets: # simple approach for now
        df['regionName'] = 'Combined'
        
    authority_list_top_n = []
    if top_n is None:
        df_filtered = df.copy()
    else:
        for region in df['regionName'].unique():
            df_filtered = df[df['regionName']==region]
            if (not df_filtered.empty) and (sort_field in df_filtered):
                df_filtered_daily, df_filtered_totals = df_get_daily_average_and_totals(df_filtered, index_col_list=['areaName'], metric=confirmed_field)
                if not df_filtered_daily.empty:
                    df_filtered_totals = df_filtered_totals.sort_values(by=sort_field, ascending=False)
                    df_filtered_totals = df_filtered_totals[:top_n]
                    authority_top_n = list(df_filtered_totals['areaName'].unique())
                    authority_list_top_n = authority_list_top_n + authority_top_n
        df_filtered = df[df['areaName'].isin(authority_list_top_n)] # would like to put it in the order of highest totals

    chart = alt.Chart()
    df_daily, df_totals = df_get_daily_average_and_totals(df_filtered, index_col_list=['areaName'], metric=confirmed_field)
    if not df_daily.empty:
        y_max = df_filtered[confirmed_field].max()
        total_max = df_totals[confirmed_field].max()
    
        trend_chart_list = []
        totals_chart_list = []
        for region in df['regionName'].unique():
            df_facet = df_filtered[df_filtered['regionName']==region]
            sorted_order = df_facet.sort_values(by=sort_field, ascending=False)['areaName'].unique()
            trend_region = trend_chart(df_facet, metric=confirmed_field, category='areaName',
                                       x_min=x_min, x_max=x_max, y_min=0, y_max=y_max, plot_7_day_average=plot_7_day_average,
                                       chart_width=trend_chart_width, chart_height=chart_height, 
                                       remove_trailing_zeros=True, 
                                       area_chart=False, row_title='', column_title=region)
            totals_region = bar_chart(df_facet, metric=confirmed_field, category='areaName',  sort_field=sort_field,
                                      rollup='total', numbers_overlaid=False,
                                      chart_width=totals_chart_width, chart_height=chart_height, 
                                      row_title='', column_title=region)

            trend_chart_list.append(trend_region)
            totals_chart_list.append(totals_region)

        trend_facet_chart = facet_wrap(trend_chart_list, plots_per_row=plots_per_row).properties(title='Daily Cases'+normalised_title_string)
        totals_facet_chart = facet_wrap(totals_chart_list, plots_per_row=plots_per_row).properties(title='Total Cases'+normalised_title_string)

        chart = trend_facet_chart | totals_facet_chart

    return chart

include_uk_local_authorities_absolute = False
if not include_uk_local_authorities_absolute:
    print('Not configured to show UK local authorities')
else:
    footer = f'Area charts and bar charts from daily data; line charts from 7 day rolling mean; highest {top_n} authorities by total confirmed cases'

    chart = uk_regions_trend_dashboard_charts(df_uk_ltla_regions, normalised=False, top_n=top_n, chart_height=chart_height, trend_chart_width=trend_chart_width, totals_chart_width=totals_chart_width)
    chart.display()
    print(footer)

    if export_slides:
        chart.save(saved_image_name)

        # Add a new slide ready for chart and set title text
        slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
        prs.set_text(slide, placeholder_name='title', text=slide_title)
        prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
        prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
slide_title = 'England Lower Tier Authorities per 100k Population'
display_h1(slide_title)

include_uk_local_authorities_per_population = True
if not include_uk_local_authorities_per_population:
    print('Not configured to show UK local authorities')
else:
    chart_height = 250
    trend_chart_width = 350
    totals_chart_width = 200

    footer = f'Area charts and bar charts from daily data; line charts from 7 day rolling mean; highest {top_n} authorities by total confirmed cases'

    chart = uk_regions_trend_dashboard_charts(df_uk_ltla_regions, normalised=True, top_n=top_n, chart_height=chart_height, trend_chart_width=trend_chart_width, totals_chart_width=totals_chart_width)
    chart.display()
    print(footer)

    if export_slides:
        chart.save(saved_image_name)

        # Add a new slide ready for chart and set title text
        slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
        prs.set_text(slide, placeholder_name='title', text=slide_title)
        prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
        prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
slide_title = 'England Selected Areas'
display_h1(slide_title)

include_england_selected_areas = False
if not include_england_selected_areas:
    print('Not configured to show England selected areas')
else:
    chart_height = 300
    trend_chart_width = 1000
    totals_chart_width = 600

    footer = f'Area charts and bar charts from daily data; line charts from 7 day rolling mean'

    filter_area_name_list=['New Forest', 'South Cambridgeshire', 'Cambridge', 'Southampton', 'Huntingdonshire', 'East Suffolk', 'Leicester']
    chart_abs = uk_regions_trend_dashboard_charts(df_uk_ltla_regions, filter_area_name_list=filter_area_name_list, normalised=False, multiple_facets=False, top_n=top_n, chart_height=chart_height, trend_chart_width=trend_chart_width, totals_chart_width=totals_chart_width)
    chart_norm = uk_regions_trend_dashboard_charts(df_uk_ltla_regions, filter_area_name_list=filter_area_name_list, normalised=True, multiple_facets=False, top_n=top_n, chart_height=chart_height, trend_chart_width=trend_chart_width, totals_chart_width=totals_chart_width)
    chart = chart_abs & chart_norm
    chart.display()
    print(footer)

    if export_slides:
        chart.save(saved_image_name)

        # Add a new slide ready for chart and set title text
        slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
        prs.set_text(slide, placeholder_name='title', text=slide_title)
        prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
        prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
slide_title = 'Covid-19 US'
display_h1(slide_title)

top_n = 20

def df_us_time_series_summary(df):
    df_totals = df.copy().drop(['country', 'state'], axis=1)

    df_totals = df_totals.groupby(['date']).agg({'lat':'mean', 'long':'mean', 
                                                 'confirmed':'sum', 'deaths':'sum', 
                                                 'Population':'sum'})
    df_totals['areaType'] = 'US'
    df_totals = df_totals.reset_index()

    return df_totals

def df_states_time_series_summary(df):
    df_totals = df.copy()
    df_totals = df_totals.groupby(['date', 'state']).agg({'lat':'mean', 'long':'mean', 
                                                              'confirmed':'sum', 'deaths':'sum', 
                                                              'Population':'sum'})

    df_totals = df_totals.reset_index()

    return df_totals

def get_us_overview_chart(df, top_n = 20):
    df = df.copy()
    
    chart_width_large = 350
    chart_width_small = 200
    chart_width_smaller = 100
    chart_height_large = 350
    chart_height = 100

    x_min = df['date'].min()
    x_max = df['date'].max()
    
    sort_field = 'deaths'
    if not top_n is None:
        df_totals = df.copy().groupby(['state']).agg({'confirmed':'sum', 'deaths':'sum'}).reset_index()
        df_totals = df_totals.sort_values(by=sort_field, ascending=False)
        df_totals = df_totals[:top_n]
        state_top_n = df_totals['state']
        df_filtered = df[df['state'].isin(state_top_n)] # would like to put it in the order of highest totals
    else:
        df_filtered = df

    df_us = df_us_time_series_summary(df)
    df_states = df_states_time_series_summary(df_filtered)


    trend_confirmed_cases  = trend_chart(df_us, metric='confirmed', category=None, x_min=x_min, x_max=x_max, 
                                         chart_width=chart_width_large, chart_height=chart_height, 
                                         row_title='US', column_title='Daily Cases')
    trend_confirmed_cases_recent  = trend_chart(df_us, metric='confirmed', category=None, x_min=x_min, x_max=x_max, filter_last_weeks=4,
                                         chart_width=chart_width_smaller, chart_height=chart_height, 
                                         row_title='', column_title='Cases 4wks')
    trend_confirmed_deaths = trend_chart(df_us, metric='deaths',    category=None, x_min=x_min, x_max=x_max, 
                                         chart_width=chart_width_large, chart_height=chart_height, 
                                         row_title='', column_title='Daily Deaths')
    trend_confirmed_deaths_recent = trend_chart(df_us, metric='deaths',    category=None, x_min=x_min, x_max=x_max, filter_last_weeks=4,
                                         chart_width=chart_width_smaller, chart_height=chart_height, 
                                         row_title='', column_title='Deaths 4wks')
    total_confirmed_cases  = bar_chart(df_us, metric='confirmed', category='areaType', rollup='total', numbers_overlaid=True,
                                         chart_width=chart_width_small, chart_height=chart_height, 
                                         row_title='', column_title='Total Cases')
    total_confirmed_deaths  = bar_chart(df_us, metric='deaths',   category='areaType', rollup='total', numbers_overlaid=True,
                                         chart_width=chart_width_small, chart_height=chart_height, 
                                         row_title='', column_title='Total Deaths')
    trend_row = trend_confirmed_cases | trend_confirmed_cases_recent | trend_confirmed_deaths | trend_confirmed_deaths_recent | total_confirmed_cases | total_confirmed_deaths
    
    trend_confirmed_cases   = trend_chart(df_states, metric='confirmed', category='state', x_min=x_min, x_max=x_max, 
                                          chart_width=chart_width_large, chart_height=chart_height,
                                          area_chart=False, row_title='States', column_title='Daily Cases')
    trend_confirmed_cases_recent   = trend_chart(df_states, metric='confirmed', category='state', x_min=x_min, x_max=x_max, filter_last_weeks=4,
                                          chart_width=chart_width_smaller, chart_height=chart_height,
                                          area_chart=False, row_title='', column_title='Cases 4wks')
    trend_confirmed_deaths  = trend_chart(df_states, metric='deaths',    category='state', x_min=x_min, x_max=x_max, 
                                          remove_trailing_zeros=True,
                                          chart_width=chart_width_large, chart_height=chart_height,
                                          area_chart=False, row_title='', column_title='Daily Deaths')
    trend_confirmed_deaths_recent  = trend_chart(df_states, metric='deaths',    category='state', x_min=x_min, x_max=x_max, filter_last_weeks=4,
                                          remove_trailing_zeros=True,
                                          chart_width=chart_width_smaller, chart_height=chart_height,
                                          area_chart=False, row_title='', column_title='Deaths 4wks')
    latest_confirmed_cases  = bar_chart(df_us, metric='confirmed', category='areaType', rollup='latest', numbers_overlaid=True,
                                          chart_width=chart_width_small, chart_height=chart_height, 
                                          row_title='', column_title='Latest Cases')
    latest_confirmed_deaths  = bar_chart(df_us, metric='deaths',   category='areaType', rollup='latest', numbers_overlaid=True,
                                          chart_width=chart_width_small, chart_height=chart_height, 
                                          row_title='', column_title='Latest Deaths')
    states_row = trend_confirmed_cases | trend_confirmed_cases_recent | trend_confirmed_deaths | trend_confirmed_deaths_recent | total_confirmed_cases | total_confirmed_deaths
    
    total_confirmed_cases  = bar_chart(df_states, metric='confirmed', category='state', rollup='total', numbers_overlaid=False,
                                         chart_width=chart_width_small, chart_height=chart_height_large, 
                                       row_title='', column_title='Total Cases')
    total_confirmed_deaths  = bar_chart(df_states, metric='deaths',   category='state', rollup='total', numbers_overlaid=False,
                                         chart_width=chart_width_small, chart_height=chart_height_large, 
                                        row_title='', column_title='Total Deaths')
    latest_confirmed_cases  = bar_chart(df_states, metric='confirmed', category='state', rollup='latest', numbers_overlaid=False,
                                         chart_width=chart_width_small, chart_height=chart_height_large, 
                                       row_title='', column_title='Latest Cases')
    latest_confirmed_deaths  = bar_chart(df_states, metric='deaths',   category='state', rollup='latest', numbers_overlaid=False,
                                         chart_width=chart_width_small, chart_height=chart_height_large, 
                                        row_title='', column_title='Latest Deaths')
    latest_row = total_confirmed_cases | total_confirmed_deaths | latest_confirmed_cases | latest_confirmed_deaths
    
    chart = trend_row & states_row & latest_row
    
    chart = configure_trend_chart(chart)

    return chart

footer = 'Area charts and bar charts from daily data; line charts from 7 day rolling mean'

chart = get_us_overview_chart(df_covid_time_series_us, top_n = top_n)
chart.display()
print(footer)

if export_slides:
    chart.save(saved_image_name)

    # Add a new slide ready for chart and set title text
    slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
    prs.set_text(slide, placeholder_name='title', text=slide_title)
    prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
    prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
slide_title = 'Covid-19 US - States'
display_h1(slide_title)

chart_width_small = 120
chart_height = 120

top_n = 20

def trend_chart_us_states(df, scale_type='linear', x_metric='date', metric='deaths', sort_field='deaths', x_min=None, x_max=None, row_title='', column_title=''):
    df = df.copy()
    df = df.reset_index()

    x_metric_type = ':T' # for date
    if x_metric == 'days_since_100':
        df = df[df['days_since_100']>=0]
        x_metric_type = ':Q' # for count of days
        x_min = 0 # no need for negative x-axis
        
    df = df.groupby(['state', 'date']).agg({'confirmed':'sum', 'deaths':'sum'})
    df = df.reset_index()

    df['weekAvg'] = df.groupby('state')[metric].rolling(window=7, min_periods=1).mean().reset_index(0, drop=True)
    
    y_max = df[metric].max()
    y_min = df[metric].min()
    if scale_type == 'log':
        y_min = max(y_min, 0.1)
        small_non_zero_value = 0.001
        df.loc[df['weekAvg']==0, 'weekAvg'] = small_non_zero_value # to enable log calculation without errors

    chart = alt.Chart().mark_area(
    ).encode(
        x   = alt.X('date:T', title=None),
        y   = metric+':Q',
        tooltip=[alt.Tooltip('date:T'),
                 alt.Tooltip(metric+':Q', format=',d')]
    ).properties(height=chart_height, width=chart_width_small)

    line = alt.Chart().mark_line(
        color='red',
    ).encode(
        x='date:T',
#         y=alt.Y('weekAvg:Q', title=row_title, scale=alt.Scale(type=scale_type, domain=[y_min, y_max], clamp=True)),
        y=alt.Y('weekAvg:Q', title=row_title, scale=alt.Scale(type=scale_type, clamp=True)),
        tooltip=[alt.Tooltip('date:T'),
                 alt.Tooltip('weekAvg:Q', format=',d')]
    )

    chart = alt.layer(chart, line, data=df).facet(alt.Column('state:N', title=None), columns=5).resolve_scale(y='independent')
        
    chart = chart.properties(title=column_title)

    return chart

def configure_trend_chart_us_states(chart):
    chart = chart.configure_headerRow(
        title=None,
    )
    
    chart = chart.configure_header(
        labelFontSize=16
    )

    chart = chart.configure_axisY(
        titleColor='blue',
        titleFontSize=20,
    )
    
    chart = chart.configure_title(
        fontSize=20,
        anchor='middle',
        color='blue'
    )
    
    return chart

def us_states_chart_summary(df, top_n = 20, sort_field = 'deaths', scale_type = 'linear'):
    df = df.copy()
    
    if not top_n is None:
        df_totals = df.copy().groupby(['state']).agg({'confirmed':'sum', 'deaths':'sum'}).reset_index()
        df_totals = df_totals.sort_values(by=sort_field, ascending=False)
        df_totals = df_totals[:top_n]
        state_top_n = df_totals['state']
        df = df[df['state'].isin(state_top_n)] # would like to put it in the order of highest totals

    us_states_trend_confirmed = trend_chart_us_states(df, scale_type=scale_type, metric='confirmed', sort_field='confirmed', row_title='', column_title='Daily Cases')
    us_states_trend_deaths    = trend_chart_us_states(df, scale_type=scale_type, metric='deaths', sort_field='deaths', row_title='', column_title='Daily Deaths')

    chart = alt.hconcat(us_states_trend_confirmed, us_states_trend_deaths).configure_concat(spacing=100)
    chart = configure_trend_chart_us_states(chart)
    
    return chart
    
footer = f'Area charts and bar charts from daily data; line charts from 7 day rolling mean; highest {top_n} states by total deaths'

chart = us_states_chart_summary(df_covid_time_series_us, top_n = top_n, sort_field = 'deaths')
chart.display()
print(footer)

if export_slides:
    chart.save(saved_image_name)

    # Add a new slide ready for chart and set title text
    slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
    prs.set_text(slide, placeholder_name='title', text=slide_title)
    prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
    prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
slide_title = 'Trend Charts - Selected Countries'
display_h1(slide_title)
small_subset = True
measure_daily = True
independent_y_scales = True

if small_subset:
    Combined_Key_list = ['India', 'Brazil', 'Sweden', 'United Kingdom', 'US', 'Ireland', 'Israel']

    print('Country list:', Combined_Key_list, latest_date_str)
    df_covid_time_series_filtered = df_covid_time_series_world[df_covid_time_series_world['Combined_Key'].isin(Combined_Key_list)].copy()
else:
    df_covid_time_series_filtered = df_covid_time_series_world.copy()

def trend_country_dashboard_charts(df, top_n=None, category=None):
    df = df.copy()
    
    chart_width_large = 350
    chart_width_small = 150
    chart_width_smaller = 110
    chart_height = 80
    
    sort_field = 'deaths'
    
    df_totals = get_totals_from_time_series(df)
    df_continents = df_continents_time_series_summary(df)

    chart = alt.vconcat()
    first_row = True
    x_min = df['date'].min()
    x_max = df['date'].max()
    
    deaths_max = df_totals['deaths'].max()
    cases_max = df_totals['confirmed'].max()
    
    for category_item in df[category].unique():
        df_filtered = df[df[category]==category_item]
        if not top_n is None:
            df_totals_filtered = df_totals.copy()
            df_totals_filtered = df_totals_filtered[df_totals_filtered[category]==category_item]
            df_totals_filtered = df_totals_filtered.sort_values(by=sort_field, ascending=False)
            df_totals_filtered = df_totals_filtered[:top_n]
            combined_key_top_n = df_totals_filtered['Combined_Key']
            df_filtered = df_filtered[df_filtered['Combined_Key'].isin(combined_key_top_n)] # would like to put it in the order of highest totals

        trend_confirmed_cases  = trend_chart(df_filtered, metric='confirmed', category=category, x_min=x_min, x_max=x_max, 
                                             chart_width=chart_width_large, chart_height=chart_height, 
                                             area_chart=True, y_range_from_avg=False,
                                             row_title=category_item, column_title='Daily Cases' if first_row else '')
        trend_confirmed_cases_recent  = trend_chart(df_filtered, metric='confirmed', category=category, x_min=x_min, x_max=x_max, filter_last_weeks=4,
                                             chart_width=chart_width_smaller, chart_height=chart_height, 
                                             row_title='', column_title='Cases 4wks' if first_row else '')
        trend_confirmed_deaths = trend_chart(df_filtered, metric='deaths',    category=category, x_min=x_min, x_max=x_max, 
                                             chart_width=chart_width_large, chart_height=chart_height, 
                                             area_chart=True, y_range_from_avg=False,
                                             row_title='', column_title='Daily Deaths' if first_row else '')
        trend_confirmed_deaths_recent = trend_chart(df_filtered, metric='deaths',    category=category, x_min=x_min, x_max=x_max, filter_last_weeks=4,
                                             chart_width=chart_width_smaller, chart_height=chart_height, 
                                             area_chart=True, y_range_from_avg=False,
                                             row_title='', column_title='Deaths 4wks' if first_row else '')

        total_confirmed_cases  = bar_chart(df_filtered, metric='confirmed', category=category, rollup='total', numbers_overlaid=False,
                                             x_min=0, x_max=cases_max,
                                             chart_width=chart_width_small, chart_height=chart_height, 
                                           row_title='', column_title='Total Cases' if first_row else '')
        total_confirmed_deaths  = bar_chart(df_filtered, metric='deaths',   category=category, rollup='total', numbers_overlaid=False,
                                             x_min=0, x_max=deaths_max,
                                             chart_width=chart_width_small, chart_height=chart_height, 
                                            row_title='', column_title='Total Deaths' if first_row else '')

        first_row = False

        row = trend_confirmed_cases | trend_confirmed_cases_recent | trend_confirmed_deaths | trend_confirmed_deaths_recent | total_confirmed_cases | total_confirmed_deaths
        chart = chart & row

    chart = chart.resolve_scale(color='independent')
    chart = configure_trend_chart(chart)
    
    return chart

footer = 'Blue area chart is daily data; red line is 7 day rolling mean'

chart = trend_country_dashboard_charts(df_covid_time_series_filtered, category='Combined_Key')
chart.display()
print(footer)

if export_slides:
    chart.save(saved_image_name)

    # Add a new slide ready for chart and set title text
    slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
    prs.set_text(slide, placeholder_name='title', text=slide_title)
    prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
    prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
current_year = 2020
clip_positive_only = True

# load country information
# https://www.mortality.org/
# https://www.mortality.org/Public/STMF_DOC/STMFNote.pdf
# https://www.mortality.org/Public/HMD-countries-codes.pdf
def load_short_term_mortality_timeseries_detailed():
    url = 'https://www.mortality.org/Public/STMF/Outputs/stmf.csv'
    df = pd.read_csv(url)
    
    # Drop redundant first row and set column names
    df = df.reset_index()
    df.columns = df.iloc[0]
    df = df.drop(df.index[0])

    # Set data types
    df['Week'] = df['Week'].astype(int)
    df['Year'] = df['Year'].astype(int)
    df['DTotal'] = df['DTotal'].astype(float).astype(int)
    df[['D0_14', 'D15_64', 'D65_74', 'D75_84', 'D85p']] = df[['D0_14', 'D15_64', 'D65_74', 'D75_84', 'D85p']].astype(float)
    df['RTotal'] = df['RTotal'].astype(float)

    # - UK England & Wales Total Population GBRTENW
    # - DE Total population DEUTNP
    country_code_replace_dict = {'DEUTNP': 'DEU', 'GBRTENW': 'GBR', 'FRATNP': 'FRA'}
    df['CountryCode'] = df['CountryCode'].replace(country_code_replace_dict)
    country_code_to_name_dict = pycountry_convert.map_country_alpha3_to_country_name()
    country_code_to_name_dict['GBR_SCO'] = 'Scotland'
    country_code_to_name_dict['GBR_NIR'] = 'Northern Ireland'
    country_code_to_name_dict['NZL_NP'] = 'New Zealand'
    df['Country'] = df['CountryCode'].map(country_code_to_name_dict)
    
    country_unknown = df[df['Country'].isnull()]['CountryCode'].drop_duplicates()
    if len(country_unknown):
        print('Unknown country codes:', country_unknown)

    return df

def load_short_term_mortality_data_detailed():
    df = load_short_term_mortality_timeseries_detailed()
    
    df = df.set_index(['Country', 'Year', 'Week', 'Sex'])
    df = df[~(df.index.get_level_values('Sex')=='b')] # drop the combined figure for both sexes
    
    df = df.filter(['D0_14', 'D15_64', 'D65_74', 'D75_84', 'D85p'])
    df = df.rename(columns={'D0_14':'Age 0-14', 'D15_64':'Age 15-64', 'D65_74':'Age 65-74', 'D75_84':'Age 75-84', 'D85p':'Age 85+'})

    df = df.reset_index()
#     df = pd.melt(df, id_vars=['Continent', 'Country', 'Year', 'Week', 'Sex'], var_name='AgeRange', value_name='Value')
    df = pd.melt(df, id_vars=['Country', 'Year', 'Week', 'Sex'], var_name='AgeRange', value_name='Value')
#     df = df.set_index(['Continent', 'Country', 'Year', 'Week', 'Sex', 'AgeRange'])
    df = df.set_index(['Country', 'Year', 'Week', 'Sex', 'AgeRange'])
    
    df_duplicated = df.loc[df.index.duplicated(keep=False)]
    if len(df_duplicated)>0:
        print('Duplicated index rows:')
        print(df_duplicated)
#         df = df.loc[~df.index.duplicated(keep='first')] # remove any duplicate index rows (e.g. for Vela at multiple frequencies)

    df = df.unstack(level=['AgeRange', 'Sex'])
    df = df.sort_index(axis='columns', level='AgeRange')

    return df

def get_excess_deaths_detailed(df, current_year, clip_positive_only):
    df = df.copy()
    
    df_previous_years = df.copy()
    df_previous_years = df_previous_years[df_previous_years.index.get_level_values('Year')<current_year]
    df_previous_years = df_previous_years.groupby(level=df_previous_years.index.names.difference(['Year']), axis=0).mean()

    df_excess_deaths = df.copy()
    df_excess_deaths = df_excess_deaths[df_excess_deaths.index.get_level_values('Year')==current_year]
    df_excess_deaths = df_excess_deaths.reset_index(level='Year', drop=True) # don't need this any more
    df_excess_deaths = df_excess_deaths - df_previous_years
    
    df_excess_deaths = df_excess_deaths.clip(lower=0) if clip_positive_only else df_excess_deaths
    
    return df_excess_deaths

def get_totals_from_time_series_detailed(df):
    df_totals = df.copy().groupby(['Country']).sum()

    return df_totals

display_h1('Mortality Analysis')

df_stmf_detailed = load_short_term_mortality_data_detailed()
df_excess_deaths_detailed = get_excess_deaths_detailed(df_stmf_detailed, current_year=current_year, clip_positive_only=clip_positive_only)
df_excess_deaths_totals_detailed = get_totals_from_time_series_detailed(df_excess_deaths_detailed)

df = df_stmf_detailed.reset_index()
print('Countries available:', df['Country'].unique())

In [ ]:
# independent_y_scales = True
# chart_height = 100
# chart_width = 200
# current_year = 2020
# clip_positive_only = True

# def trend_chart_deaths_overall(df, current_year, scale_type='linear', metric='DTotal', row_title='', column_title='', row_height=200, chart_width=200):
#     df = df.copy()
    
#     week_max = df[df['Year']==current_year]['Week'].max()
#     week_trust_margin = 3
#     week_num_trust = max(week_max-week_trust_margin, 0)
    
#     y_min = 0 # max(df[metric].min(), 0)
#     y_max = df[metric].max()
    
#     line = alt.Chart().mark_line( #mark_area
#     ).transform_filter(
#         alt.datum.Year < 2020
#     ).encode(
#         x=alt.X('Week:Q'),
#         y=alt.Y(f'mean({metric}):Q', title=row_title, scale=alt.Scale(domain=[y_min, y_max], type=scale_type)),
# #         color=alt.Color(f'{category}:N', sort=None, legend=None),
#         tooltip=[alt.Tooltip(f'mean({metric}):Q', format=',d')]
#     )
    
#     line_2020 = alt.Chart().mark_line(
#         strokeDash=[5,5],
#     ).transform_filter(
#         (alt.datum.Year == 2020) &
# #         (alt.datum.Sex == sex) & 
#         (alt.datum.Week <= week_num_trust),
#     ).encode(
#         x=alt.X('Week:Q'),
#         y=alt.Y(f'mean({metric}):Q'),
# #         color=alt.Color(f'{category}:N', sort=None, legend=None),
#         tooltip=[alt.Tooltip('Week:Q'),
#                  alt.Tooltip(f'{metric}:Q', format=',d')]
#     )

#     line_2020_recent = alt.Chart().mark_line(
#         strokeDash=[1,3],
#     ).transform_filter(
#         (alt.datum.Year == 2020) &
# #         (alt.datum.Sex == sex) & 
#         (alt.datum.Week >= week_num_trust),
#     ).encode(
#         x=alt.X('Week:Q'),
#         y=alt.Y(f'mean({metric}):Q'),
# #         color=alt.Color(f'{category}:N', sort=None, legend=None),
#         tooltip=[alt.Tooltip('Week:Q'),
#                  alt.Tooltip(f'{metric}:Q', format=',d')]
#     )

#     band = alt.Chart().mark_area(
#         opacity = 0.3,
#     ).transform_filter(
#         alt.datum.Year < 2020
#     ).encode(
#         x=alt.X('Week:Q'),
#         y=alt.Y(f'min({metric}):Q'),
#         y2=alt.Y2(f'max({metric}):Q'),
# #         color=alt.Color(f'{category}:N', sort=None, legend=None),
#         tooltip=[alt.Tooltip('Week:Q'),
#                  alt.Tooltip(f'max({metric}):Q', title='Max', format=',d'),
#                  alt.Tooltip(f'mean({metric}):Q', title='Mean', format=',d'),
#                  alt.Tooltip(f'min({metric}):Q', title='Min', format=',d')]
#     )
    
#     chart = alt.layer(line, band, line_2020, line_2020_recent, data=df)
#     chart = chart.properties(title=column_title, height=row_height, width=chart_width)

#     return chart

# # def trend_chart_deaths_age_range(df, current_year, category='Country', sex='', scale_type='linear', metric='DTotal', row_title='', column_title='', row_height=200, chart_width=200):
# #     df = df.copy()
# #     df = df.reset_index()
    
# #     week_max = df[df['Year']==current_year]['Week'].max()
# #     week_trust_margin = 3
# #     week_num_trust = max(week_max-week_trust_margin, 0)
    
# #     y_min = max(df[metric].min(), 0)
# #     y_max = df[metric].max()
    
# #     line = alt.Chart().mark_line( #mark_area
# #     ).transform_filter(
# #         alt.datum.Year < 2020
# #     ).encode(
# #         x=alt.X('Week:Q'),
# #         y=alt.Y(f'mean({metric}):Q', title=row_title, scale=alt.Scale(domain=[y_min, y_max], type=scale_type)),
# #         color=alt.Color(f'{category}:N', sort=None, legend=None),
# #         tooltip=[alt.Tooltip(f'{category}:N'),
# #                  alt.Tooltip(f'mean({metric}):Q', format=',d')]
# #     )
    
# #     line_2020 = alt.Chart().mark_line(
# #         strokeDash=[5,5],
# #     ).transform_filter(
# #         (alt.datum.Year == 2020) &
# #         (alt.datum.Sex == sex) & 
# #         (alt.datum.Week <= week_num_trust),
# #     ).encode(
# #         x=alt.X('Week:Q'),
# #         y=alt.Y(f'mean({metric}):Q'),
# #         color=alt.Color(f'{category}:N', sort=None, legend=None),
# #         tooltip=[alt.Tooltip(f'{category}:N'),
# #                  alt.Tooltip('Week:Q'),
# #                  alt.Tooltip(f'{metric}:Q', format=',d')]
# #     )

# #     line_2020_recent = alt.Chart().mark_line(
# #         strokeDash=[1,3],
# #     ).transform_filter(
# #         (alt.datum.Year == 2020) &
# #         (alt.datum.Sex == sex) & 
# #         (alt.datum.Week >= week_num_trust),
# #     ).encode(
# #         x=alt.X('Week:Q'),
# #         y=alt.Y(f'mean({metric}):Q'),
# #         color=alt.Color(f'{category}:N', sort=None, legend=None),
# #         tooltip=[alt.Tooltip(f'{category}:N'),
# #                  alt.Tooltip('Week:Q'),
# #                  alt.Tooltip(f'{metric}:Q', format=',d')]
# #     )

# #     band = alt.Chart().mark_area(
# #         opacity = 0.3,
# #     ).transform_filter(
# #         alt.datum.Year < 2020
# #     ).encode(
# #         x=alt.X('Week:Q'),
# #         y=alt.Y(f'min({metric}):Q'),
# #         y2=alt.Y2(f'max({metric}):Q'),
# #         color=alt.Color(f'{category}:N', sort=None, legend=None),
# #         tooltip=[alt.Tooltip(f'{category}:N'),
# #                  alt.Tooltip('Week:Q'),
# #                  alt.Tooltip(f'max({metric}):Q', title='Max', format=',d'),
# #                  alt.Tooltip(f'mean({metric}):Q', title='Mean', format=',d'),
# #                  alt.Tooltip(f'min({metric}):Q', title='Min', format=',d')]
# #     )
    
# #     chart = alt.layer(line, band, line_2020, line_2020_recent, data=df)
# #     chart = chart.properties(title=column_title, height=row_height, width=chart_width)

# #     return chart

# def trend_chart_excess_deaths_age_range(df, category='Country', sex='', scale_type='linear', metric='DTotal', row_title='', column_title='', row_height=200, chart_width=200):
#     df = df.copy()
#     df = df.reset_index()
    
#     week_max = df['Week'].max()
#     week_trust_margin = 3
#     week_num_trust = max(week_max-week_trust_margin, 0)

#     y_min = df[metric].min()
#     y_max = df[metric].max()
    
#     line = alt.Chart().mark_line(
#         strokeDash=[5,5],
#     ).transform_filter(
#         (alt.datum.Sex == sex) & 
#         (alt.datum.Week <= week_num_trust),
#     ).encode(
#         x=alt.X('Week:Q'),
#         y=alt.Y(f'{metric}:Q', title=row_title, scale=alt.Scale(domain=[y_min, y_max], type=scale_type)),
#         color=alt.Color(f'{category}:N', sort=None, legend=None),
#         tooltip=[alt.Tooltip(f'{category}:N'),
#                  alt.Tooltip(f'{metric}:Q', format=',d')]
#     )
        
#     line_recent = alt.Chart().mark_line(
#         strokeDash=[1,3],
#     ).transform_filter(
#         (alt.datum.Sex == sex) & 
#         (alt.datum.Week >= week_num_trust),
#     ).encode(
#         x=alt.X('Week:Q'),
#         y=alt.Y(f'{metric}:Q', title=row_title, scale=alt.Scale(domain=[y_min, y_max], type=scale_type)),
#         color=alt.Color(f'{category}:N', sort=None, legend=None),
#         tooltip=[alt.Tooltip(f'{category}:N'),
#                  alt.Tooltip(f'{metric}:Q', format=',d')]
#     )

#     chart = alt.layer(line, line_recent, data=df)
#     chart = chart.properties(title=column_title, height=row_height, width=chart_width)

#     return chart

# # category: 'continent', 'country'
# def get_bar_chart_coloured_age_range(df, category='Country', sex='', scale_type='linear', metric='DTotal', row_title='', column_title='', x_min=None, x_max=None, row_height=200, chart_width=200):
#     x_min = 0 if not x_min else x_min
#     x_max = df[metric].max() if not x_max else x_max
    
#     bars = alt.Chart(df).mark_bar().transform_filter(
#         (alt.datum.Sex == sex)
#     ).encode(
#         x=alt.X(f'{metric}:Q', title=metric, scale=alt.Scale(domain=[x_min, x_max], type=scale_type)),
#         y=alt.Y(f'{category}:N', sort=None, title=row_title), # sort='-x' to sort locally in this chart
#         color=alt.Color(f'{category}:N', sort=None, legend=None),
#         tooltip=[alt.Tooltip(f'{category}:N'),
#                  alt.Tooltip(f'{metric}:Q', format=',d')]
#     ).properties(
#         height=row_height, width=chart_width,
#     )

#     text = bars.mark_text(
#         align='left',
#         baseline='middle',
#         dx=3  # Nudges text to right so it doesn't appear on top of the bar
#     ).encode(
#         text=alt.Text(metric+':Q', format=',d') #,.2r gives rounded to 2 significant figures
#     )
    
#     chart = (bars + text)
#     chart = chart.properties(title=column_title)

#     return chart

# # category: 'continent', 'country'
# def get_bar_chart_coloured_overall(df, scale_type='linear', metric='DTotal', row_title='', column_title='', x_min=None, x_max=None, row_height=200, chart_width=200):
#     x_min = 0 if not x_min else x_min
#     x_max = df[metric].max() if not x_max else x_max
    
#     bars = alt.Chart(df).mark_bar().encode(
#         x=alt.X(f'{metric}:Q', title=metric, scale=alt.Scale(domain=[x_min, x_max], type=scale_type)),
# #         y=alt.Y(category+':N', sort=None, title=row_title), # sort='-x' to sort locally in this chart
# #         color=alt.Color(f'{category}:N', sort=None, legend=None),
#         tooltip=[alt.Tooltip(f'{metric}:Q', format=',d')]
#     ).properties(
#         height=row_height, width=chart_width,
#     )

#     text = bars.mark_text(
#         align='left',
#         baseline='middle',
#         dx=3  # Nudges text to right so it doesn't appear on top of the bar
#     ).encode(
#         text=alt.Text(f'{metric}:Q', format=',d') #,.2r gives rounded to 2 significant figures
#     )
    
#     chart = (bars + text)
#     chart = chart.properties(title=column_title)

#     return chart

# def trend_dashboard_charts_age_range(df, df_excess, df_totals, current_year, clip_positive_only, top_n=None, metric='Value', scale_type='linear', country_filter_list=None, row_height=200, chart_width=200):
#     df_overall = df.copy()
#     df_overall = df_overall.stack(['AgeRange', 'Sex'])
#     df_overall = df_overall.groupby(level=df_overall.index.names.difference(['AgeRange', 'Sex']), axis=0).sum()
#     df_overall = df_overall.reset_index()
#     df_overall = df_overall[df_overall['Country'].isin(country_filter_list)] if country_filter_list else df_overall
    
#     df = df.copy()
#     df = df.stack(['AgeRange', 'Sex'])
#     df = df.reset_index()
#     df = df[df['Country'].isin(country_filter_list)] if country_filter_list else df
    
#     df_excess = df_excess.copy()
#     df_excess = df_excess.stack(['AgeRange', 'Sex'])
#     df_excess = df_excess.clip(lower=0) if clip_positive_only else df_excess
#     df_excess = df_excess.reset_index()
#     df_excess = df_excess[df_excess['Country'].isin(country_filter_list)] if country_filter_list else df_excess

#     df_totals = df_totals
#     df_totals = df_totals.stack(['AgeRange', 'Sex'])
#     df_totals = df_totals.clip(lower=0) if clip_positive_only else df_totals
#     df_totals_overall = df_totals.groupby(level=df_totals.index.names.difference(['AgeRange', 'Sex']), axis=0).sum()
#     df_totals = df_totals.reset_index()
#     df_totals_overall = df_totals_overall.reset_index()
#     df_totals = df_totals[df_totals['Country'].isin(country_filter_list)] if country_filter_list else df_totals
    
#     chart = alt.vconcat()
#     first_row = True
    
#     category='Country'
    
#     excess_deaths_max_per_category = None #df_totals[metric].max()
#     excess_deaths_overall_max = df_totals_overall[metric].max()

#     country_list = df['Country'].unique()
#     for country in country_list:
#         df_overall_filtered = df_overall[df_overall['Country']==country]
#         df_filtered = df[df['Country']==country]
#         df_excess_filtered = df_excess[df_excess['Country']==country]
#         df_totals_filtered = df_totals[df_totals['Country']==country]
#         df_totals_overall_filtered = df_totals_overall[df_totals_overall['Country']==country]

#         trend_deaths_overall = trend_chart_deaths_overall(df_overall_filtered, current_year=current_year, metric=metric,
#                                           row_title=country, column_title='Deaths' if first_row else '',
#                                           row_height=row_height, chart_width=chart_width)

#         trend_excess_deaths_male = trend_chart_excess_deaths_age_range(df_excess_filtered, category='AgeRange', sex='m', metric=metric,
#                                                         row_title='', column_title='Excess Deaths Male' if first_row else '',
#                                                         row_height=row_height, chart_width=chart_width)

#         trend_excess_deaths_female = trend_chart_excess_deaths_age_range(df_excess_filtered, category='AgeRange', sex='f', metric=metric,
#                                                         row_title='', column_title='Excess Deaths Female' if first_row else '',
#                                                         row_height=row_height, chart_width=chart_width)

#         chart_deaths_male    = get_bar_chart_coloured_age_range(df_totals_filtered, category='AgeRange', sex='m', metric=metric, 
#                                                         row_title='', column_title='Total Excess Deaths Male' if first_row else '',
#                                                         x_min=0, x_max=excess_deaths_max_per_category,
#                                                         row_height=row_height, chart_width=chart_width)
        
#         chart_deaths_female    = get_bar_chart_coloured_age_range(df_totals_filtered, category='AgeRange', sex='f', metric=metric, 
#                                                         row_title='', column_title='Total Excess Deaths Female' if first_row else '',
#                                                         x_min=0, x_max=excess_deaths_max_per_category,
#                                                         row_height=row_height, chart_width=chart_width)

#         chart_deaths_overall    = get_bar_chart_coloured_overall(df_totals_overall_filtered, metric=metric, 
#                                                         row_title='', column_title='Total Excess Deaths' if first_row else '',
#                                                         x_min=0, x_max=excess_deaths_overall_max,
#                                                         row_height=row_height, chart_width=chart_width)

#         first_row = False

#         row = trend_deaths_overall | trend_excess_deaths_male | trend_excess_deaths_female | chart_deaths_male | chart_deaths_female | chart_deaths_overall
#         chart = (chart & row).resolve_scale(color='independent')
    
#     return chart


# def configure_trend_chart_continents(chart):
#     chart = chart.configure_axisY(
#         titleColor='blue',
#         titleFontSize=20,
#     )
#     chart = chart.configure_axisX(
# #         title=None
#     )

#     chart = chart.configure_title(
#         fontSize=20,
#         anchor='middle',
#         color='blue'
#     )
#     return chart

# country_filter_list = ['Spain', 'Sweden', 'United Kingdom', 'United States', 'Israel'] # 'Russian Federation'

# slide_title = 'Excess Deaths '+str(current_year) + ' - Selected Countries'
# display_h2(slide_title)
# print('Dashed line is for current year; dotted is for last 3 weeks since data might not yet be final')
# print('Shaded area is min/max range for previous years, with mean as solid line')
# print('Where current year is significantly lower than previous years, it is likely to be due to data not yet available')

# footer = 'Dashed line is current year; dotted is last 3wks (might not be complete); shaded is min/max for previous years, with mean as solid line'
# chart = trend_dashboard_charts_age_range(df_stmf_detailed, df_excess_deaths_detailed, df_excess_deaths_totals_detailed, 
#                                          current_year=current_year, clip_positive_only=clip_positive_only,
#                                          country_filter_list=country_filter_list,
#                                          row_height=chart_height, chart_width=chart_width)
# chart = configure_trend_chart_continents(chart)

# chart.display()
# print(footer)

# if export_slides:
#     chart.save(saved_image_name)

#     # Add a new slide ready for chart and set title text
#     slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
#     prs.set_text(slide, placeholder_name='title', text=slide_title)
#     prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
#     prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
# independent_y_scales = True
# chart_height = 350
# chart_width = 150
# current_year = 2020
# clip_positive_only = True

# def trend_chart_deaths_overall_cumulative(df, current_year, scale_type='linear', metric='DTotal', row_title='', column_title='', row_height=200, chart_width=200):
#     df = df.copy()
    
#     df['cumulative'] = df.groupby(['Year'])[metric].apply(lambda x: x.cumsum())
#     metric='cumulative'
    
#     week_max = df[df['Year']==current_year]['Week'].max()
#     week_trust_margin = 3
#     week_num_trust = max(week_max-week_trust_margin, 0)
    
#     y_min = 0 # max(df[metric].min(), 0)
#     y_max = df[metric].max()
    
#     line = alt.Chart().mark_line(
#     ).transform_filter(
#         alt.datum.Year < 2020
#     ).encode(
#         x=alt.X('Week:Q'),
#         y=alt.Y(f'mean({metric}):Q', title=row_title, scale=alt.Scale(domain=[y_min, y_max], type=scale_type)),
# #         color=alt.Color(f'{category}:N', sort=None, legend=None),
#         tooltip=[alt.Tooltip(f'mean({metric}):Q', format=',d')]
#     )
    
#     line_2020 = alt.Chart().mark_line(
#         strokeDash=[5,5],
#     ).transform_filter(
#         (alt.datum.Year == 2020) &
# #         (alt.datum.Sex == sex) & 
#         (alt.datum.Week <= week_num_trust),
#     ).encode(
#         x=alt.X('Week:Q'),
#         y=alt.Y(f'mean({metric}):Q'),
# #         color=alt.Color(f'{category}:N', sort=None, legend=None),
#         tooltip=[alt.Tooltip('Week:Q'),
#                  alt.Tooltip(f'{metric}:Q', format=',d')]
#     )

#     line_2020_recent = alt.Chart().mark_line(
#         strokeDash=[1,3],
#     ).transform_filter(
#         (alt.datum.Year == 2020) &
# #         (alt.datum.Sex == sex) & 
#         (alt.datum.Week >= week_num_trust),
#     ).encode(
#         x=alt.X('Week:Q'),
#         y=alt.Y(f'mean({metric}):Q'),
# #         color=alt.Color(f'{category}:N', sort=None, legend=None),
#         tooltip=[alt.Tooltip('Week:Q'),
#                  alt.Tooltip(f'{metric}:Q', format=',d')]
#     )

#     band = alt.Chart().mark_area(
#         opacity = 0.3,
#     ).transform_filter(
#         alt.datum.Year < 2020
#     ).encode(
#         x=alt.X('Week:Q'),
#         y=alt.Y(f'min({metric}):Q'),
#         y2=alt.Y2(f'max({metric}):Q'),
# #         color=alt.Color(f'{category}:N', sort=None, legend=None),
#         tooltip=[alt.Tooltip('Week:Q'),
#                  alt.Tooltip(f'max({metric}):Q', title='Max', format=',d'),
#                  alt.Tooltip(f'mean({metric}):Q', title='Mean', format=',d'),
#                  alt.Tooltip(f'min({metric}):Q', title='Min', format=',d')]
#     )
    
#     chart = alt.layer(line, band, line_2020, line_2020_recent, data=df)
#     chart = chart.properties(title=column_title, height=row_height, width=chart_width)

#     return chart

# def trend_dashboard_charts_cumulative(df, df_excess, df_totals, current_year, clip_positive_only, top_n=None, metric='Value', scale_type='linear', country_filter_list=None, row_height=200, chart_width=200):
#     df_overall = df.copy()
#     df_overall = df_overall.stack(['AgeRange', 'Sex'])
#     df_overall = df_overall.groupby(level=df_overall.index.names.difference(['AgeRange', 'Sex']), axis=0).sum()
#     df_overall = df_overall.reset_index()
#     df_overall = df_overall[df_overall['Country'].isin(country_filter_list)] if country_filter_list else df_overall
    
#     df = df.copy()
#     df = df.stack(['AgeRange', 'Sex'])
#     df = df.reset_index()
#     df = df[df['Country'].isin(country_filter_list)] if country_filter_list else df

#     chart = alt.hconcat()
#     first_row = True
    
#     category='Country'
    
#     excess_deaths_max_per_category = None #df_totals[metric].max()

#     country_list = df['Country'].unique()
#     for country in country_list:
#         df_overall_filtered = df_overall[df_overall['Country']==country]

#         trend_deaths_overall = trend_chart_deaths_overall_cumulative(df_overall_filtered, current_year=current_year, metric=metric,
#                                           row_title=country, column_title='Deaths' if first_row else '',
#                                           row_height=row_height, chart_width=chart_width)

#         first_row = False

#         chart = chart | trend_deaths_overall
    
#     return chart


# country_filter_list = ['Spain', 'Sweden', 'United Kingdom', 'United States', 'Israel'] # 'Russian Federation'

# slide_title = 'Cumulative Deaths '+str(current_year) + ' - Selected Countries'
# display_h2(slide_title)
# print('Dashed line is for current year; dotted is for last 3 weeks since data might not yet be final')
# print('Shaded area is min/max range for previous years, with mean as solid line')
# print('Where current year is significantly lower than previous years, it is likely to be due to data not yet available')

# footer = 'Dashed line is current year; dotted is last 3wks (might not be complete); shaded is min/max for previous years, with mean as solid line'
# chart = trend_dashboard_charts_cumulative(df_stmf_detailed, df_excess_deaths_detailed, df_excess_deaths_totals_detailed, 
#                                          current_year=current_year, clip_positive_only=clip_positive_only,
#                                          country_filter_list=country_filter_list,
#                                          row_height=chart_height, chart_width=chart_width)
# chart = configure_trend_chart_continents(chart)

# chart.display()
# print(footer)

# if export_slides:
#     chart.save(saved_image_name)

#     # Add a new slide ready for chart and set title text
#     slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
#     prs.set_text(slide, placeholder_name='title', text=slide_title)
#     prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
#     prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
since_year = 2020

def get_excess_deaths_detailed_multi_year(df, since_year, clip_positive_only):
    df = df.copy()
    
    df_previous_years = df.copy()
    df_previous_years = df_previous_years[df_previous_years.index.get_level_values('Year')<since_year]
    df_previous_years = df_previous_years.groupby(level=df_previous_years.index.names.difference(['Year']), axis=0).mean()

    df_excess_deaths = df.copy()
    df_excess_deaths = df_excess_deaths[df_excess_deaths.index.get_level_values('Year')>=since_year]
    
    years = sorted(df_excess_deaths.index.get_level_values('Year').unique())
    df_excess_deaths_multi_year = pd.DataFrame()
    for year in years:
        df2 = df_excess_deaths[df_excess_deaths.index.get_level_values('Year')==year]
        df2 = df2.reset_index(level='Year', drop=True)
        df2 = df2 - df_previous_years
        df2 = df2.clip(lower=0) if clip_positive_only else df2
        df2['Year'] = year
        df2 = df2.set_index(['Year'], append=True)
        df_excess_deaths_multi_year = pd.concat([df_excess_deaths_multi_year, df2])

    return df_excess_deaths_multi_year

def get_totals_from_time_series_detailed_multi_year(df):
    df_totals = df.copy().groupby(['Country']).sum()

    return df_totals

display_h1('Mortality Analysis - since '+str(since_year))

# df_stmf_detailed = load_short_term_mortality_data_detailed()
df_stmf_detailed_multi_year = df_stmf_detailed.copy()

since_year = 2020

df_excess_deaths_detailed_multi_year = get_excess_deaths_detailed_multi_year(df_stmf_detailed, since_year=since_year, clip_positive_only=clip_positive_only)
df_excess_deaths_totals_detailed_multi_year = get_totals_from_time_series_detailed_multi_year(df_excess_deaths_detailed_multi_year)

# country_filter_list = ['Spain', 'Sweden', 'United Kingdom', 'United States', 'Israel'] # 'Russian Federation'
# df = df_excess_deaths_detailed_multi_year
# df = df.reset_index()
# df = df[df['Country'].isin(country_filter_list)] if country_filter_list else df
# display(df)

In [ ]:
independent_y_scales = True
chart_height = 100
chart_width = 200
since_year = 2020
clip_positive_only = True

def trend_chart_deaths_overall_multi_year(df, since_year, scale_type='linear', metric='DTotal', row_title='', column_title='', row_height=200, chart_width=200):
    df = df.copy()
    
    latest_year = df['Year'].max()

    week_max = df[df['Year']==latest_year]['Week'].max()
    week_trust_margin = 3
    week_num_trust = max(week_max-week_trust_margin, 0)
    
    
    df_previous_years = df[df['Year'] < since_year].copy()
    df_previous_years = df_previous_years.groupby(['Week']).agg({metric: ['min', 'mean', 'max']})
    df_previous_years.columns = df_previous_years.columns.droplevel(0)
#     display(df_previous_years.head())
    
    df = df[df['Year'] >= since_year].copy()
    df = pd.merge(df, df_previous_years, on='Week')
#     display(df.head())
    
    df['ISO'] = df['Year'].astype(str) + '-W' + df['Week'].astype(str) + '-1'
    df['date'] = df['ISO'].map(lambda x: datetime.datetime.strptime(x, "%G-W%V-%u"))
    df = df.drop(['ISO'], axis='columns')
#     display(df.head())
    
    y_min = 0 # max(df[metric].min(), 0)
    y_max = df[metric].max()
    
    line = alt.Chart().mark_line( #mark_area
#     ).transform_filter(
#         alt.datum.Year < since_year
    ).encode(
        x=alt.X('date:T'),
#         y=alt.Y(f'mean({metric}):Q', title=row_title, scale=alt.Scale(domain=[y_min, y_max], type=scale_type)),
        y=alt.Y('mean:Q', title=row_title, scale=alt.Scale(domain=[y_min, y_max], type=scale_type)),
#         color=alt.Color(f'{category}:N', sort=None, legend=None),
        tooltip=[alt.Tooltip('mean:Q', format=',d')]
    )
    
    line_since_year = alt.Chart().mark_line(
        strokeDash=[5,5],
    ).transform_filter(
        (alt.datum.Year < latest_year) |
#         (alt.datum.Sex == sex) & 
        (alt.datum.Week <= week_num_trust),
    ).encode(
        x=alt.X('date:T'),
        y=alt.Y(f'mean({metric}):Q'),
#         color=alt.Color(f'{category}:N', sort=None, legend=None),
        tooltip=[alt.Tooltip('Year:Q'),
                 alt.Tooltip('Week:Q'),
                 alt.Tooltip(f'{metric}:Q', format=',d')]
    )

    line_recent = alt.Chart().mark_line(
        strokeDash=[1,3],
    ).transform_filter(
        (alt.datum.Year == latest_year) &
#         (alt.datum.Sex == sex) & 
        (alt.datum.Week >= week_num_trust),
    ).encode(
#         x=alt.X('Week:Q'),
        x=alt.X('date:T'),
        y=alt.Y(f'mean({metric}):Q'),
#         color=alt.Color(f'{category}:N', sort=None, legend=None),
        tooltip=[alt.Tooltip('Year:Q'),
                 alt.Tooltip('Week:Q'),
                 alt.Tooltip(f'{metric}:Q', format=',d')]
    )

    band = alt.Chart().mark_area(
        opacity = 0.3,
#     ).transform_filter(
#         alt.datum.Year < since_year
    ).encode(
        x=alt.X('date:T'),
#         y=alt.Y(f'min({metric}):Q'),
#         y2=alt.Y2(f'max({metric}):Q'),
        y=alt.Y('min:Q'),
        y2=alt.Y2('max:Q'),
#         color=alt.Color(f'{category}:N', sort=None, legend=None),
        tooltip=[alt.Tooltip('Year:Q'),
                 alt.Tooltip('Week:Q'),
                 alt.Tooltip('max:Q', title='Max', format=',d'),
                 alt.Tooltip('mean:Q', title='Mean', format=',d'),
                 alt.Tooltip('min:Q', title='Min', format=',d')]
    )
    
    chart = alt.layer(line, band, line_since_year, line_recent, data=df)
    chart = chart.properties(title=column_title, height=row_height, width=chart_width)

    return chart

def trend_chart_excess_deaths_age_range_multi_year(df, category='Country', sex='', scale_type='linear', metric='DTotal', row_title='', column_title='', row_height=200, chart_width=200):
    df = df.copy()
    df = df.reset_index()
    
    latest_year = df['Year'].max()

    week_max = df[df['Year']==latest_year]['Week'].max()
    week_trust_margin = 3
    week_num_trust = max(week_max-week_trust_margin, 0)
    

    df['ISO'] = df['Year'].astype(str) + '-W' + df['Week'].astype(str) + '-1'
    df['date'] = df['ISO'].map(lambda x: datetime.datetime.strptime(x, "%G-W%V-%u"))
    df = df.drop(['ISO'], axis='columns')
#     display(df.head())

    y_min = df[metric].min()
    y_max = df[metric].max()
    
    line = alt.Chart().mark_line(
        strokeDash=[5,5],
    ).transform_filter(
        (alt.datum.Sex == sex) #& 
#         (alt.datum.Week <= week_num_trust),
    ).encode(
        x=alt.X('date:T'),
        y=alt.Y(f'{metric}:Q', title=row_title, scale=alt.Scale(domain=[y_min, y_max], type=scale_type)),
        color=alt.Color(f'{category}:N', sort=None, legend=None),
        tooltip=[alt.Tooltip(f'{category}:N'),
                 alt.Tooltip(f'{metric}:Q', format=',d')]
    )
        
    line_recent = alt.Chart().mark_line(
        strokeDash=[1,3],
    ).transform_filter(
        (alt.datum.Sex == sex) & 
        (alt.datum.Year == latest_year) &
        (alt.datum.Week >= week_num_trust),
    ).encode(
        x=alt.X('date:T'),
        y=alt.Y(f'{metric}:Q', title=row_title, scale=alt.Scale(domain=[y_min, y_max], type=scale_type)),
        color=alt.Color(f'{category}:N', sort=None, legend=None),
        tooltip=[alt.Tooltip(f'{category}:N'),
                 alt.Tooltip(f'{metric}:Q', format=',d')]
    )

    chart = alt.layer(line, line_recent, data=df)
    chart = chart.properties(title=column_title, height=row_height, width=chart_width)

    return chart

# category: 'continent', 'country'
def get_bar_chart_coloured_age_range(df, category='Country', sex='', scale_type='linear', metric='DTotal', row_title='', column_title='', x_min=None, x_max=None, row_height=200, chart_width=200):
    x_min = 0 if not x_min else x_min
    x_max = df[metric].max() if not x_max else x_max
    
    bars = alt.Chart(df).mark_bar().transform_filter(
        (alt.datum.Sex == sex)
    ).encode(
        x=alt.X(f'{metric}:Q', title=metric, scale=alt.Scale(domain=[x_min, x_max], type=scale_type)),
        y=alt.Y(f'{category}:N', sort=None, title=row_title), # sort='-x' to sort locally in this chart
        color=alt.Color(f'{category}:N', sort=None, legend=None),
        tooltip=[alt.Tooltip(f'{category}:N'),
                 alt.Tooltip(f'{metric}:Q', format=',d')]
    ).properties(
        height=row_height, width=chart_width,
    )

    text = bars.mark_text(
        align='left',
        baseline='middle',
        dx=3  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
        text=alt.Text(metric+':Q', format=',d') #,.2r gives rounded to 2 significant figures
    )
    
    chart = (bars + text)
    chart = chart.properties(title=column_title)

    return chart

# category: 'continent', 'country'
def get_bar_chart_coloured_overall(df, scale_type='linear', metric='DTotal', row_title='', column_title='', x_min=None, x_max=None, row_height=200, chart_width=200):
    x_min = 0 if not x_min else x_min
    x_max = df[metric].max() if not x_max else x_max
    
    bars = alt.Chart(df).mark_bar().encode(
        x=alt.X(f'{metric}:Q', title=metric, scale=alt.Scale(domain=[x_min, x_max], type=scale_type)),
#         y=alt.Y(category+':N', sort=None, title=row_title), # sort='-x' to sort locally in this chart
#         color=alt.Color(f'{category}:N', sort=None, legend=None),
        tooltip=[alt.Tooltip(f'{metric}:Q', format=',d')]
    ).properties(
        height=row_height, width=chart_width,
    )

    text = bars.mark_text(
        align='left',
        baseline='middle',
        dx=3  # Nudges text to right so it doesn't appear on top of the bar
    ).encode(
        text=alt.Text(f'{metric}:Q', format=',d') #,.2r gives rounded to 2 significant figures
    )
    
    chart = (bars + text)
    chart = chart.properties(title=column_title)

    return chart

def configure_trend_chart_continents(chart):
    chart = chart.configure_axisY(
        titleColor='blue',
        titleFontSize=20,
    )
    chart = chart.configure_axisX(
#         title=None
    )

    chart = chart.configure_title(
        fontSize=20,
        anchor='middle',
        color='blue'
    )
    return chart

def trend_dashboard_charts_age_range_multi_year(df, df_excess, df_totals, since_year, clip_positive_only, top_n=None, metric='Value', scale_type='linear', country_filter_list=None, row_height=200, chart_width=200):
#     display(df.head())
    df_overall = df.copy()
    df_overall = df_overall.stack(['AgeRange', 'Sex'])
    df_overall = df_overall.groupby(level=df_overall.index.names.difference(['AgeRange', 'Sex']), axis=0).sum()
    df_overall = df_overall.reset_index()
    df_overall = df_overall[df_overall['Country'].isin(country_filter_list)] if country_filter_list else df_overall
    
    df = df.copy()
    df = df.stack(['AgeRange', 'Sex'])
    df = df.reset_index()
    df = df[df['Country'].isin(country_filter_list)] if country_filter_list else df
    
    df_excess = df_excess.copy()
    df_excess = df_excess.stack(['AgeRange', 'Sex'])
    df_excess = df_excess.clip(lower=0) if clip_positive_only else df_excess
    df_excess = df_excess.reset_index()
    df_excess = df_excess[df_excess['Country'].isin(country_filter_list)] if country_filter_list else df_excess

    df_totals = df_totals
    df_totals = df_totals.stack(['AgeRange', 'Sex'])
    df_totals = df_totals.clip(lower=0) if clip_positive_only else df_totals
    df_totals_overall = df_totals.groupby(level=df_totals.index.names.difference(['AgeRange', 'Sex']), axis=0).sum()
    df_totals = df_totals.reset_index()
    df_totals_overall = df_totals_overall.reset_index()
    df_totals = df_totals[df_totals['Country'].isin(country_filter_list)] if country_filter_list else df_totals
    
    chart = alt.vconcat()
    first_row = True
    
    category='Country'
    
    excess_deaths_max_per_category = None #df_totals[metric].max()
    excess_deaths_overall_max = df_totals_overall[metric].max()

    country_list = df['Country'].unique()
    for country in country_list:
        df_overall_filtered = df_overall[df_overall['Country']==country]
        df_filtered = df[df['Country']==country]
        df_excess_filtered = df_excess[df_excess['Country']==country]
        df_totals_filtered = df_totals[df_totals['Country']==country]
        df_totals_overall_filtered = df_totals_overall[df_totals_overall['Country']==country]
        
#         display(df_overall_filtered.head())

        trend_deaths_overall = trend_chart_deaths_overall_multi_year(df_overall_filtered, since_year=since_year, metric=metric,
                                          row_title=country, column_title='Deaths' if first_row else '',
                                          row_height=row_height, chart_width=chart_width)

        trend_excess_deaths_male = trend_chart_excess_deaths_age_range_multi_year(df_excess_filtered, category='AgeRange', sex='m', metric=metric,
                                                        row_title='', column_title='Excess Deaths Male' if first_row else '',
                                                        row_height=row_height, chart_width=chart_width)

        trend_excess_deaths_female = trend_chart_excess_deaths_age_range_multi_year(df_excess_filtered, category='AgeRange', sex='f', metric=metric,
                                                        row_title='', column_title='Excess Deaths Female' if first_row else '',
                                                        row_height=row_height, chart_width=chart_width)

        chart_deaths_male    = get_bar_chart_coloured_age_range(df_totals_filtered, category='AgeRange', sex='m', metric=metric, 
                                                        row_title='', column_title='Total Excess Deaths Male' if first_row else '',
                                                        x_min=0, x_max=excess_deaths_max_per_category,
                                                        row_height=row_height, chart_width=chart_width)
        
        chart_deaths_female    = get_bar_chart_coloured_age_range(df_totals_filtered, category='AgeRange', sex='f', metric=metric, 
                                                        row_title='', column_title='Total Excess Deaths Female' if first_row else '',
                                                        x_min=0, x_max=excess_deaths_max_per_category,
                                                        row_height=row_height, chart_width=chart_width)

        chart_deaths_overall    = get_bar_chart_coloured_overall(df_totals_overall_filtered, metric=metric, 
                                                        row_title='', column_title='Total Excess Deaths' if first_row else '',
                                                        x_min=0, x_max=excess_deaths_overall_max,
                                                        row_height=row_height, chart_width=chart_width)

        first_row = False

        row = trend_deaths_overall | trend_excess_deaths_male | trend_excess_deaths_female | chart_deaths_male | chart_deaths_female | chart_deaths_overall
        chart = (chart & row).resolve_scale(color='independent')
    
    return chart

country_filter_list = ['Spain', 'Sweden', 'United Kingdom', 'United States', 'Israel'] # 'Russian Federation'

slide_title = 'Excess Deaths - Since '+str(since_year) + ' - Selected Countries'
display_h2(slide_title)
print(f'Dashed line is since {since_year}; dotted is for last 3 weeks since data might not yet be final')
print('Shaded area is min/max range for previous years, with mean as solid line')
print('Where current year is significantly lower than previous years, it is likely to be due to data not yet available')

footer = f'Dashed line is since {since_year}; dotted is last 3wks (might not be complete); shaded is min/max for previous years, with mean as solid line'

chart = trend_dashboard_charts_age_range_multi_year(df_stmf_detailed_multi_year, df_excess_deaths_detailed_multi_year, df_excess_deaths_totals_detailed_multi_year, 
                                         since_year=since_year, clip_positive_only=clip_positive_only,
                                         country_filter_list=country_filter_list,
                                         row_height=chart_height, chart_width=chart_width)
chart = configure_trend_chart_continents(chart)

chart.display()
print(footer)

if export_slides:
    chart.save(saved_image_name)

    # Add a new slide ready for chart and set title text
    slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
    prs.set_text(slide, placeholder_name='title', text=slide_title)
    prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
    prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
independent_y_scales = True
chart_height = 350
chart_width = 150
since_year = 2020
clip_positive_only = True

def trend_chart_deaths_overall_cumulative_multi_year(df, since_year, scale_type='linear', metric='DTotal', row_title='', column_title='', row_height=200, chart_width=200):
    df = df.copy()
    
#     df['cumulative'] = df.groupby(['Year'])[metric].apply(lambda x: x.cumsum())
#     metric='cumulative'
    
    latest_year = df['Year'].max()

    week_max = df[df['Year']==latest_year]['Week'].max()
    week_trust_margin = 3
    week_num_trust = max(week_max-week_trust_margin, 0)
    

    
    
    df_previous_years = df[df['Year'] < since_year].copy()
    previous_years_list = df_previous_years['Year'].unique()

#     df_previous_years['cumulative'] = df.groupby(['Year'])[metric].apply(lambda x: x.cumsum())
    df_previous_years = df_previous_years.pivot(index='Week', columns='Year')[metric]
#     df_previous_years.columns = pd.MultiIndex.from_product([['years'], df_previous_years.columns])
#     display(df_previous_years.head())
    
#     df_previous_years = df_previous_years.groupby(['Week']).agg({metric: ['min', 'mean', 'max']})
#     df_previous_years = df_previous_years.groupby(['Week']).agg({'cumulative': ['min', 'mean', 'max']})
#     df_previous_years.columns = df_previous_years.columns.droplevel(0) # remove 'value' from multi-level column names from groupby function
#     display(df_previous_years.head())
    
    df = df[df['Year'] >= since_year].copy()
#     df.columns = pd.MultiIndex.from_product([['Original'], df.columns])
    df = pd.merge(df, df_previous_years, on='Week')
    df = df.sort_values(by=['Year', 'Week'], ascending=True)
    df[metric] = df.groupby(['Country'])[metric].apply(lambda x: x.cumsum())
    df[previous_years_list] = df.groupby(['Country'])[previous_years_list].apply(lambda x: x.cumsum())
    df['min'] = df[previous_years_list].min(axis='columns')
    df['mean'] = df[previous_years_list].mean(axis='columns')
    df['max'] = df[previous_years_list].max(axis='columns')
    df = df.drop(previous_years_list, axis='columns')
#     display(df.head())
    
    df['ISO'] = df['Year'].astype(str) + '-W' + df['Week'].astype(str) + '-1'
    df['date'] = df['ISO'].map(lambda x: datetime.datetime.strptime(x, "%G-W%V-%u"))
    df = df.drop(['ISO'], axis='columns')
#     display(df.head())


#     df = df.sort_values(by=['date'], ascending=True)
    
#     df['min'] = df.groupby(['Country'])['min'].apply(lambda x: x.cumsum())
#     df['mean'] = df.groupby(['Country'])['mean'].apply(lambda x: x.cumsum())
#     df['max'] = df.groupby(['Country'])['max'].apply(lambda x: x.cumsum())
#     metric='cumulative'
    
#     display(df)



    
    y_min = 0 # max(df[metric].min(), 0)
    y_max = df[metric].max()
    
    line = alt.Chart().mark_line(
#     ).transform_filter(
#         alt.datum.Year < since_year
    ).encode(
#         x=alt.X('Week:Q'),
        x=alt.X('date:T'),
        y=alt.Y('mean:Q', title=row_title, scale=alt.Scale(domain=[y_min, y_max], type=scale_type)),
#         color=alt.Color(f'{category}:N', sort=None, legend=None),
        tooltip=[alt.Tooltip('mean:Q', format=',d')]
    )
    
    line_since_year = alt.Chart().mark_line(
        strokeDash=[5,5],
    ).transform_filter(
        (alt.datum.Year < latest_year) |
#         (alt.datum.Sex == sex) & 
        (alt.datum.Week <= week_num_trust),
    ).encode(
#         x=alt.X('Week:Q'),
        x=alt.X('date:T'),
        y=alt.Y(f'mean({metric}):Q'),
#         color=alt.Color(f'{category}:N', sort=None, legend=None),
        tooltip=[alt.Tooltip('date:T'),
                 alt.Tooltip(f'{metric}:Q', format=',d')]
    )

    line_recent = alt.Chart().mark_line(
        strokeDash=[1,3],
    ).transform_filter(
        (alt.datum.Year == latest_year) &
#         (alt.datum.Sex == sex) & 
        (alt.datum.Week >= week_num_trust),
    ).encode(
#         x=alt.X('Week:Q'),
        x=alt.X('date:T'),
        y=alt.Y(f'mean({metric}):Q'),
#         color=alt.Color(f'{category}:N', sort=None, legend=None),
        tooltip=[alt.Tooltip('date:T'),
                 alt.Tooltip(f'{metric}:Q', format=',d')]
    )

    band = alt.Chart().mark_area(
        opacity = 0.3,
#     ).transform_filter(
#         alt.datum.Year < 2020
    ).encode(
#         x=alt.X('Week:Q'),
        x=alt.X('date:T'),
        y=alt.Y('min:Q'),
        y2=alt.Y2('max:Q'),
#         color=alt.Color(f'{category}:N', sort=None, legend=None),
        tooltip=[alt.Tooltip('Week:Q'),
                 alt.Tooltip('max:Q', title='Max', format=',d'),
                 alt.Tooltip(f'mean({metric}):Q', title='Mean', format=',d'),
                 alt.Tooltip('min:Q', title='Min', format=',d')]
    )
    
    chart = alt.layer(line, band, line_since_year, line_recent, data=df)
#     chart = alt.layer(line, band, line_recent, data=df)
    chart = chart.properties(title=column_title, height=row_height, width=chart_width)

    return chart

def trend_dashboard_charts_cumulative_multi_year(df, df_excess, df_totals, since_year, clip_positive_only, top_n=None, metric='Value', scale_type='linear', country_filter_list=None, row_height=200, chart_width=200):
    df_overall = df.copy()
    df_overall = df_overall.stack(['AgeRange', 'Sex'])
    df_overall = df_overall.groupby(level=df_overall.index.names.difference(['AgeRange', 'Sex']), axis=0).sum()
    df_overall = df_overall.reset_index()
    df_overall = df_overall[df_overall['Country'].isin(country_filter_list)] if country_filter_list else df_overall
    
    df = df.copy()
    df = df.stack(['AgeRange', 'Sex'])
    df = df.reset_index()
    df = df[df['Country'].isin(country_filter_list)] if country_filter_list else df

    chart = alt.hconcat()
    first_row = True
    
    category='Country'
    
    excess_deaths_max_per_category = None #df_totals[metric].max()

    country_list = df['Country'].unique()
    for country in country_list:
        df_overall_filtered = df_overall[df_overall['Country']==country]

        trend_deaths_overall = trend_chart_deaths_overall_cumulative_multi_year(df_overall_filtered, since_year=since_year, metric=metric,
                                          row_title=country, column_title='Deaths' if first_row else '',
                                          row_height=row_height, chart_width=chart_width)

        first_row = False

        chart = chart | trend_deaths_overall
    
    return chart


country_filter_list = ['Spain', 'Sweden', 'United Kingdom', 'United States', 'Israel'] # 'Russian Federation'

slide_title = 'Cumulative Deaths - Since '+str(since_year) + ' - Selected Countries'
display_h2(slide_title)
print(f'Dashed line is since {since_year}; dotted is for last 3 weeks since data might not yet be final')
print('Shaded area is min/max range for previous years, with mean as solid line')
print('Where current year is significantly lower than previous years, it is likely to be due to data not yet available')

footer = f'Dashed line is since {since_year}; dotted is last 3wks (might not be complete); shaded is min/max for previous years, with mean as solid line'
chart = trend_dashboard_charts_cumulative_multi_year(df_stmf_detailed_multi_year, df_excess_deaths_detailed_multi_year, df_excess_deaths_totals_detailed_multi_year, 
                                         since_year=since_year, clip_positive_only=clip_positive_only,
                                         country_filter_list=country_filter_list,
                                         row_height=chart_height, chart_width=chart_width)
chart = configure_trend_chart_continents(chart)

chart.display()
print(footer)

if export_slides:
    chart.save(saved_image_name)

    # Add a new slide ready for chart and set title text
    slide = prs.add_slide(layout = prs.slide_template_layout.TITLE_AND_MAX_CONTENT)
    prs.set_text(slide, placeholder_name='title', text=slide_title)
    prs.add_image(slide, placeholder_name='content', image_file=saved_image_name)
    prs.set_text(slide, placeholder_name='footer', text=footer)

In [ ]:
if export_slides:
    # save slides
    print('Saving slides')
    prs.save(slides_file_name)